<a href="https://colab.research.google.com/github/gonyoly/portfolio/blob/main/M_52_UK_PLACES_COMPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  PRELIMINARY PROCEDURES

In [ ]:
#@title INSTALATIONS
# !pip install numpy==1.26.0 --force-reinstall
# !pip install flair
# !pip install spacy
# !python -m spacy download uk_core_news_lg
!pip install rapidfuzz
!pip install fuzzywuzzy

# import os
# os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.6 MB/s eta 0:00:00


In [ ]:
#@title IMPORTS

import pathlib
pathlib.WindowsPath = pathlib.PosixPath

# from flair.data import Sentence, Corpus
# from flair.models import SequenceTagger
import pandas as pd
import random
import logging
import openpyxl
import tqdm
import re
import os
from google.colab import drive
import IPython.display as ipd
from IPython.display import display, HTML, Markdown
from fuzzywuzzy import fuzz, process
import subprocess
import sys
import numpy as np
import matplotlib.pyplot as plt
import json, hashlib
from collections import Counter


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#@title LOADING DATA & MODEL

# mount Google Drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"
!ls "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"

# load model
# tagger = SequenceTagger.load('lang-uk/flair-uk-ner')



Mounted at /content/drive
 admin_units_extracted_fond_and_all_title.xlsx
 admun_presence_pilot.xlsx
 COMPL_PLACES_FROM_INCOMINGS
 consolidated_data_original.xlsx
 consolidated_data_working_copy.csv
 consolidated_data_working_copy_with_title_and_provenance_places_bef_lemm.xlsx
 consolidated_data_working_copy_with_title_and_provenance_places.xlsx
 consolidated_data_working_copy.xlsx
 dana_mapping_duplicates.xlsx
'DRAFTS_DEBUGS_&_PILOTS_ARCHIVE'
 final_places_after_lemm_after_dedupl.xlsx
 final_places_after_lemm_bef_dedupl.xlsx
 final_places_mach_transl_with_fixed.xlsx
 final_places_mach_transl.xlsx
 final_places_with_sapir_take1.xlsx
 INCOMINGS
 lowercase_terms_for_lemm_dictionary.xlsx
 lowercase_terms_with_repl_patterns.xlsx
 match_update_report.xlsx
 missing_union_table.xlsx
 nonempty_consolidated_unmatched.xlsx
 nonempty_master_unmatched.xlsx
 OLD_IRRELEVANT
 PLACE_EXTRACTION_TAKE_1
 Place_ID_final.txt
 places_missing_in_sapir_administrative_division.xlsx
 places_missing_in_sapir_dist

In [ ]:
#@title PROCESSING AND FILTERING INCOMINGS FOR PLACE COLUMNS


# loading the mapping table
mapping_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/t_metadata_mapping.xlsx"
mapping_df = pd.read_excel(mapping_path)

# filtering relevant column names
relevant_columns = mapping_df[
    ~mapping_df["Column_Name"].str.contains("itle|fond|street|flat|house") &
    mapping_df["FIELD_NAME"].str.contains("PLACE")
]["Column_Name"].tolist()

# defining directories
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"
incomings_dir = base_dir + "/INCOMINGS"
irrelevant_dir = incomings_dir + "/IRRELEVANT_FOR_PLACES"
if not os.path.exists(irrelevant_dir):
    os.makedirs(irrelevant_dir)

# iterating over files and classifying them
files = [f for f in os.listdir(incomings_dir) if f.endswith(".xlsx")]
relevant_files = []
irrelevant_files = []
file_columns = {}
column_files = {}
column_counts = {}

for file in files:
    df = pd.read_excel(incomings_dir + "/" + file, sheet_name=0)
    found = [col for col in relevant_columns if col in df.columns]
    if found:
        relevant_files.append(file)
        file_columns[file] = found
        for col in found:
            nonnull_count = df[col].notna().sum()
            column_files.setdefault(col, []).append(file)
            column_counts[(file, col)] = nonnull_count
    else:
        irrelevant_files.append(file)
        os.rename(incomings_dir + "/" + file, irrelevant_dir + "/" + file)

# section a: print relevant tables and their relevant columns with counts
print("relevant tables:")
for file, cols in file_columns.items():
    print(f"- {file}")
    for col in cols:
        count = column_counts[(file, col)]
        print(f"    - {col} ({count})")

# section b: print relevant columns and the tables they appear in
print("\nrelevant columns:")
for col, files in column_files.items():
    print(f"- {col}")
    for file in files:
        print(f"    - {file}")

# section c: print irrelevant tables
print("\nirrelevant tables:")
for file in irrelevant_files:
    print(f"- {file}")

# section d: display 7 random non-null values for each relevant column
for col, files in column_files.items():
    # collecting all non-null values for this column across relevant files
    values = []
    for file in files:
        df = pd.read_excel(incomings_dir + "/" + file, sheet_name=0, usecols=[col])
        values.extend(df[col].dropna().tolist())
    sample_vals = pd.Series(values).sample(n=min(7, len(values)), replace=False)
    display(sample_vals.to_frame(name=col))

relevant tables:
- Incoming_13439437_meta.xlsx
    - creator (371)
- Incoming_13569102_meta.xlsx
    - Places (17)
    - Provenance (17)
- Incoming_11803151_meta.xlsx
    - en_details_mod (43)
    - en_details_mod (43)
    - ru_details_mod (43)
    - ru_details_mod (43)
- Incoming_14384469_meta.xlsx
    - description_02_en (919)
    - description_02_uk (919)
    - note_uk (6)

relevant columns:
- creator
    - Incoming_13439437_meta.xlsx
- Places
    - Incoming_13569102_meta.xlsx
- Provenance
    - Incoming_13569102_meta.xlsx
- en_details_mod
    - Incoming_11803151_meta.xlsx
    - Incoming_11803151_meta.xlsx
- ru_details_mod
    - Incoming_11803151_meta.xlsx
    - Incoming_11803151_meta.xlsx
- description_02_en
    - Incoming_14384469_meta.xlsx
- description_02_uk
    - Incoming_14384469_meta.xlsx
- note_uk
    - Incoming_14384469_meta.xlsx

irrelevant tables:
- Incoming_13493437_meta.xlsx
- Incoming_12804595_meta.xlsx
- Incoming_12801584_meta.xlsx
- Incoming_14665688_meta.xlsx
- Inco

,creator
237,УСБУ у Львівській обл.
281,УСБУ у Львівській обл.
50,УСБУ в Хмельницькій обл.
368,УСБУ у Львівській обл.
239,УСБУ у Львівській обл.
18,УСБУ в Житомирській обл.
213,УСБУ у Львівській обл.


,Places
10,"Сарненский, Степанский и Тучинский районы Рове..."
0,Винниковский район Львовская область
5,город Дрогобыч Дрогобычская область
16,"города Черновицы и Сторожинец; Вижницкий, Вашк..."
3,Львовско-Сельский район Львовская область
6,город Ровно Ровенская область
7,город Ровно и Ровенский район Ровенская область


,Provenance
5,Головний представник уряду УРСР в справах евак...
4,Головний представник уряду УРСР в справах евак...
9,Головний представник уряду УРСР в справах евак...
13,Головний представник уряду УРСР в справах евак...
1,Головний представник уряду УРСР в справах евак...
15,Головний представник уряду УРСР в справах евак...
7,Головний представник уряду УРСР в справах евак...


,en_details_mod
76,Document # 1 /Building management office #1340...
83,Document # 1 /Building management office # 130...
12,Document # 1 /The corpse of Simkin Shmul Aizik...
54,Document # 1 /Certificate /The deceased Simkin...
13,Documents # 1-2 /Number 1661 /The corpse of St...
59,Document # 1 /To Kharkov morgue /Please take t...
40,Document # 1 /Building management office # 130...


,ru_details_mod
25,Документ # 1 /Харьковская городская Управа /В ...
74,Документ # 1 /Домоуправление № 1340 /Барачный ...
44,Документ # 1 /От Полиции 6 района /города Харь...
24,Документ # 1 /Управление Бургомистра 11 района...
33,Документ # 1 /Домоуправление № 1340 /Барачный ...
83,"Документ # 1 /Домоуправление № 1305, 13 район ..."
30,"Документ # 1 /Домоуправление № 1334, 13 район ..."


,description_02_en
492,The Ternopil Regional Commission for Assistanc...
427,Department: residential
221,Department: economic
753,Organizational reports
823,Documents about the fallen underground
476,The Ternopil Regional Commission for Assistanc...
479,The Ternopil Regional Commission for Assistanc...


,description_02_uk
410,Відділ: житловий
745,Організаційні звіти
555,Тернопільська обласна комісія сприяння в робот...
452,Відділ: житловий
302,Відділ: житловий
265,Відділ: житловий
654,Документи про полеглих підпільників


,note_uk
4,ґлибочок Малий Ободівка. Гнилиці Великі. Капус...
0,' фрагмент записів ^фрагмент записів
1,* фрагмент записів з фрагмент записів
2,* фрагмент записів
3,"* Про народжен ня: листопад, грудень, жовтень ..."
5,Рос. мовою


In [ ]:
#@title MAPPING OLD ID ON GENERAL ACT TABLE

# loading general act ids table
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"
general_path = base_dir + "/t_General_Act_IDs.xlsx"
general_df = pd.read_excel(general_path, sheet_name=0)

# loading master mapping table
master_path = base_dir + "/t_Master_table_draft_dana_withdates_mapped.xlsx"
master_df = pd.read_excel(master_path, sheet_name=0)

# merging on id_master with left join
merged_df = general_df.merge(
    master_df[["ID_master", "old_id"]],
    on="ID_master",
    how="left"
)

# filling missing old_id with empty string
merged_df["old_id"] = merged_df["old_id"].fillna("")

# saving merged table to new file
output_path = base_dir + "/t_General_Act_IDs_with_old_ID.xlsx"
merged_df.to_excel(output_path, index=False)

# calculating statistics for old_id population
total = len(merged_df)
filled = merged_df["old_id"].astype(bool).sum()
empty = total - filled
filled_pct = filled / total * 100
empty_pct = empty / total * 100

# outputting counts and percentages
print(f"filled old_id: {filled} ({filled_pct:.2f}%)")
print(f"empty old_id: {empty} ({empty_pct:.2f}%)")


filled old_id: 14971 (76.45%)
empty old_id: 4613 (23.55%)


In [ ]:
#@title BUILDING COMBINED TEXT_&_MEMO DF FOR PLACE EXTRACTION

# loading mapping table and directories
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"
incomings_dir = base_dir + "/INCOMINGS"
mapping_path = base_dir + "/t_General_Act_IDs_with_old_ID.xlsx"
mapping_df = pd.read_excel(mapping_path, sheet_name=0)

# renaming mapping columns and casting to string types
mapping_df = mapping_df.rename(columns={
    "Incoming": "incoming",
    "ID": "ID",
    "Act_ID": "Act_ID",
    "Sign": "Sign"
})
mapping_df["incoming"] = mapping_df["incoming"].astype(str)
mapping_df["ID"] = mapping_df["ID"].astype(int).astype(str)
mapping_df["Act_ID"] = mapping_df["Act_ID"].astype(int).astype(str)

# defining static relevant columns for each incoming file
files_columns = {
    "Incoming_11803151_meta.xlsx": ["ru_details_mod"],
    "Incoming_13439437_meta.xlsx": ["creator"],
    "Incoming_13569102_meta.xlsx": ["Places", "Provenance"],
    "Incoming_14384469_meta.xlsx": ["description_02_uk", "note_uk"]
}

# constructing combined records list
records = []
for file, cols in files_columns.items():
    df = pd.read_excel(incomings_dir + "/" + file, sheet_name=0)
    incoming_col = next(c for c in df.columns if c.lower() == "incoming")
    id_col = next(c for c in df.columns if c.lower() == "id")
    for col in cols:
        mask = df[col].notna()
        for idx, value in df.loc[mask, col].items():
            rec = {
                "TEXT_&_MEMO_VALUES": value,
                "source": col,
                "incoming": str(int(df.at[idx, incoming_col])),
                "ID": str(int(df.at[idx, id_col])),
                "FILE_NAME": file
            }
            records.append(rec)

# creating combined dataframe
combined_df = pd.DataFrame(records)

# merging with mapping to add Act_ID and Sign
combined_df = combined_df.merge(
    mapping_df[["incoming", "ID", "Act_ID", "Sign"]],
    on=["incoming", "ID"],
    how="left"
)
combined_df[["Act_ID", "Sign"]] = combined_df[["Act_ID", "Sign"]].fillna("")

# reordering columns
combined_df = combined_df[[
    "Act_ID",
    "TEXT_&_MEMO_VALUES",
    "source",
    "incoming",
    "ID",
    "FILE_NAME",
    "Sign"
]]

# saving result to excel
output_dir = base_dir + "/COMPL_PLACES_FROM_INCOMINGS"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_path = output_dir + "/incoming_text&memo_data_for_compl_place_extraction.xlsx"
combined_df.to_excel(output_path, index=False)

# computing and printing statistics
total = len(combined_df)

# overall statistics: total rows
print("STATISTICS SUMMARY\n")
print(f"total rows: {total}\n")

# stats by incoming
print("stats by incoming:\n")
incoming_counts = combined_df["incoming"].value_counts()
for inc, cnt in incoming_counts.items():
    print(f"incoming {inc}: {cnt} ({cnt/total*100:.2f}%)")

# stats by source
print("\nstats by source:\n")
source_counts = combined_df["source"].value_counts()
for src, cnt in source_counts.items():
    print(f"source {src}: {cnt} ({cnt/total*100:.2f}%)")

# displaying sample rows for each source+incoming
samples = []
for (src, inc), group in combined_df.groupby(["source", "incoming"]):
    samples.append(group.sample(n=min(5, len(group))))
sample_df = pd.concat(samples).sort_values(by="source")
display(sample_df)


STATISTICS SUMMARY

total rows: 1373

stats by incoming:

incoming 14384469: 925 (67.37%)
incoming 13439437: 371 (27.02%)
incoming 11803151: 43 (3.13%)
incoming 13569102: 34 (2.48%)

stats by source:

source description_02_uk: 919 (66.93%)
source creator: 371 (27.02%)
source ru_details_mod: 43 (3.13%)
source Places: 17 (1.24%)
source Provenance: 17 (1.24%)
source note_uk: 6 (0.44%)


,Act_ID,TEXT_&_MEMO_VALUES,source,incoming,ID,FILE_NAME,Sign
430,1514,"города Черновицы и Сторожинец; Вижницкий, Вашк...",Places,13569102,17,Incoming_13569102_meta.xlsx,3_3229_53_00008
415,1499,Винниковский район Львовская область,Places,13569102,2,Incoming_13569102_meta.xlsx,3_3229_35_00059
424,1508,"Сарненский, Степанский и Тучинский районы Рове...",Places,13569102,11,Incoming_13569102_meta.xlsx,3_3229_48_00017
425,1510,"Вербовский, Демидовский, Острожецкий, Козински...",Places,13569102,12,Incoming_13569102_meta.xlsx,3_3229_49_00010
417,1501,Львовско-Сельский район Львовская область,Places,13569102,4,Incoming_13569102_meta.xlsx,3_3229_35_00063
447,1514,Головний представник уряду УРСР в справах евак...,Provenance,13569102,17,Incoming_13569102_meta.xlsx,3_3229_53_00008
443,1509,Головний представник уряду УРСР в справах евак...,Provenance,13569102,13,Incoming_13569102_meta.xlsx,3_3229_49_00009
440,1507,Головний представник уряду УРСР в справах евак...,Provenance,13569102,10,Incoming_13569102_meta.xlsx,3_3229_48_00016
442,1510,Головний представник уряду УРСР в справах евак...,Provenance,13569102,12,Incoming_13569102_meta.xlsx,3_3229_49_00010
433,1500,Головний представник уряду УРСР в справах евак...,Provenance,13569102,3,Incoming_13569102_meta.xlsx,3_3229_35_00062


#  EXTRACTION OF PLACES FROM TEXT_&_MEMO_VALUES (creator + description_02_uk + note_uk + Places + Provenance + ru_details_mod)

In [ ]:
#@title EXTRACTING PLACES FROM TEXT_&_MEMO_VALUES (OLD METHOD, NER ONLY)


# # loading the dataset from excel into a pandas dataframe
# df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/incoming_text&memo_data_for_compl_place_extraction.xlsx")


# # loading the flair ner model for ukrainian
# tagger = SequenceTagger.load('lang-uk/flair-uk-ner')


# # defining a function to extract location entities using flair ner
# def extract_locations(text, tagger):
#     if not isinstance(text, str):
#         return ''
#     sentence = Sentence(text)
#     tagger.predict(sentence)
#     locations = {entity.text for entity in sentence.get_spans('ner') if entity.tag == 'LOC'}
#     return ', '.join(locations)


# # inserting the extracted_places column immediately after TEXT_&_MEMO_VALUES
# df.insert(
#     df.columns.get_loc('TEXT_&_MEMO_VALUES') + 1,
#     'extracted_places',
#     df['TEXT_&_MEMO_VALUES'].apply(lambda x: extract_locations(x, tagger))
# )


# # saving the updated dataframe to a new excel file
# df.to_excel(
#     "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/incoming_text&memo_data_for_compl_place_extraction_with_places.xlsx",
#     index=False
# )


# # calculating and printing overall statistics for extracted_places
# total = len(df)
# non_empty = df['extracted_places'].astype(bool).sum()
# empty = total - non_empty
# print(f"total rows: {total}")
# print(f"non-empty extracted_places: {non_empty} ({non_empty/total*100:.2f}%)")
# print(f"empty extracted_places: {empty} ({empty/total*100:.2f}%)")


# # calculating and printing average and maximum number of places for non-empty rows
# counts = df.loc[df['extracted_places'].astype(bool), 'extracted_places'].str.count(',') + 1
# print(f"average number of places: {counts.mean():.2f}")
# print(f"max number of places: {counts.max()}")


# # calculating and printing statistics grouped by source
# for src in df['source'].unique():
#     subset = df[df['source'] == src]
#     total_src = len(subset)
#     non_empty_src = subset['extracted_places'].astype(bool).sum()
#     counts_src = subset.loc[subset['extracted_places'].astype(bool), 'extracted_places'].str.count(',') + 1
#     avg_src = counts_src.mean() if not counts_src.empty else 0
#     max_src = counts_src.max() if not counts_src.empty else 0

#     print(f"\nsource: {src}")
#     print(f" total rows: {total_src}")
#     print(f" non-empty extracted_places: {non_empty_src} ({non_empty_src/total_src*100:.2f}%)")
#     print(f" average places per non-empty: {avg_src:.2f}")
#     print(f" max places in one row: {max_src}")


# # displaying 20 random samples from the dataframe with all columns
# print("\nrandom samples:")
# display(df.sample(20))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/72.9M [00:00<?, ?B/s]

2025-04-22 10:50:53,904 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>
total rows: 1373
non-empty extracted_places: 509 (37.07%)
empty extracted_places: 864 (62.93%)
average number of places: 1.35
max number of places: 12

source: ru_details_mod
 total rows: 43
 non-empty extracted_places: 30 (69.77%)
 average places per non-empty: 1.93
 max places in one row: 5

source: creator
 total rows: 371
 non-empty extracted_places: 364 (98.11%)
 average places per non-empty: 1.00
 max places in one row: 1

source: Places
 total rows: 17
 non-empty extracted_places: 17 (100.00%)
 average places per non-empty: 4.18
 max places in one row: 12

source: Provenance
 total rows: 17
 non-empty extracted_places: 17 (100.00%)
 average places per non-empty: 1.76
 max places in one row: 2

source: description_02_uk
 total rows: 919
 non-empty extracted_places: 80 (8

,Act_ID,TEXT_&_MEMO_VALUES,extracted_places,source,incoming,ID,FILE_NAME,Sign
158,146.0,УСБУ у Львівській обл.,Львівській обл,creator,13439437,242,Incoming_13439437_meta.xlsx,1_0005_01_00147
119,NaN,УСБУ в Житомирській обл.,Житомирській обл,creator,13439437,77,Incoming_13439437_meta.xlsx,NaN
1087,7537.0,Інформативні звіти,,description_02_uk,14384469,3344,Incoming_14384469_meta.xlsx,4_R-3567_01_00053
832,6832.0,Відділ: житловий,,description_02_uk,14384469,2977,Incoming_14384469_meta.xlsx,4_R-0181_01_00324
4,8336.0,Документы # 1-2 /№ 1611 /Труп гр. Гольденберг ...,,ru_details_mod,11803151,5,Incoming_11803151_meta.xlsx,6_R-3009_10_01611
603,6591.0,Відділ: мельдунковий,,description_02_uk,14384469,2736,Incoming_14384469_meta.xlsx,4_R-0181_01_00072
44,1493.0,Невідомо. Тимчасово окупована територія,,creator,13439437,2,Incoming_13439437_meta.xlsx,1_X_01_07980
1203,7582.0,Організаційні звіти,,description_02_uk,14384469,3389,Incoming_14384469_meta.xlsx,4_R-3567_01_00098
1289,7678.0,Документи про полеглих підпільників,,description_02_uk,14384469,3485,Incoming_14384469_meta.xlsx,4_R-3567_01_00194
861,6855.0,Відділ: житловий,,description_02_uk,14384469,3000,Incoming_14384469_meta.xlsx,4_R-0181_01_00347


In [ ]:
#@title VISUALISATION OF NER IN TEXT_&_MEMO_VALUES

# loading the excel file into a pandas dataframe
df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/incoming_text&memo_data_for_compl_place_extraction.xlsx")

# loading the flair ukrainian ner model
tagger = SequenceTagger.load('lang-uk/flair-uk-ner')

# defining color mapping for each entity type
def get_color(entity_type):
    colors = {
        'PER': '#ffa500',
        'LOC': '#1e90ff',
        'ORG': '#32cd32',
        'MISC': '#ffd700'
    }
    return colors.get(entity_type, '#ffffff')

# defining a function to visualize named entities with html highlighting
def visualize_ner(text):
    text = str(text)
    sent = Sentence(text)
    tagger.predict(sent)
    html_content = ''
    last_idx = 0
    entities = sent.get_spans('ner')
    for entity in entities:
        start, end = entity.start_position, entity.end_position
        entity_text = text[start:end]
        entity_type = entity.get_label('ner').value
        color = get_color(entity_type)
        html_content += text[last_idx:start]
        html_content += f'<mark style="background-color: {color}; font-weight: bold; border-radius: 0.5em; padding: 0.1em 0.3em; margin: 0 0.25em;">{entity_text}<span style="font-size: 0.8em; font-weight: bold; vertical-align: top; margin-left: 0.5em;">{entity_type}</span></mark>'
        last_idx = end
    html_content += text[last_idx:]
    container_html = f'<div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; font-family: arial; font-size: 16px; line-height: 1.5;">{html_content}</div>'
    info_html = f'<p style="font-family: arial; font-size: 14px; color: #555;">found {len(entities)} entities.</p>'
    display(HTML(container_html + info_html))

# sampling and visualizing texts for ner demonstration
sample_size = int(len(df) * 1)
random_sample = df['TEXT_&_MEMO_VALUES'].dropna().sample(n=sample_size, random_state=42)
for txt in random_sample:
    visualize_ner(txt)

2025-04-22 11:35:29,484 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>


In [ ]:
#@title EXTRACT POTENTIAL MISRECOGNISED PLACES FOR MANUAL ANNOTATION


# loading the dataset from excel into a pandas dataframe
df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/incoming_text&memo_data_for_compl_place_extraction.xlsx")


# initializing the flair ner model
tagger = SequenceTagger.load('lang-uk/flair-uk-ner')


# defining regex pattern for capitalized words
capital_pattern = r'\b[А-ЯҐЄІЇ]+[а-яґєії]*\b'


# collecting ner-detected loc entities
ner_locs = set()
for text in df['TEXT_&_MEMO_VALUES'].dropna():
    sentence = Sentence(text)
    tagger.predict(sentence)
    for entity in sentence.get_spans('ner'):
        if entity.get_label('ner').value == 'LOC':
            ner_locs.add(entity.text)


# collecting all unique capitalized words
capital_words = set()
for text in df['TEXT_&_MEMO_VALUES'].dropna():
    matches = re.findall(capital_pattern, text)
    capital_words.update(matches)


# determining unrecognized capitalized words
unrecognized = sorted(capital_words - ner_locs)


# creating annotation dataframe
annot_df = pd.DataFrame({'capitalized_nonloc': unrecognized})
annot_df['is_place'] = ''


# saving annotation dataframe to excel
annot_df.to_excel(
    "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/potential_misrecognised_places_for_man_annotation.xlsx",
    index=False
)


# printing total number of rows
print(f"total rows: {len(annot_df)}")


# displaying 20 random samples for inspection
display(annot_df.sample(20))


2025-04-23 08:06:42,065 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>
total rows: 452


,capitalized_nonloc,is_place
97,Дмитриевская,
61,Вишгородоцька,
318,Пугачева,
1,ІНСТРУКТИВНІ,
284,Павленковский,
368,Степанского,
120,Забирать,
72,Выдана,
290,Пинхус,
156,Клесовский,


In [ ]:
#@title EXTRACTING PLACES FROM TEXT_&_MEMO_VALUES USING REGEX & NER

# loading the dataset from excel into a pandas dataframe
df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/incoming_text&memo_data_for_compl_place_extraction.xlsx")

# loading the manual annotation file with capitalized tokens marked as places
ann_df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/potential_misrecognised_places_for_man_annotation.xlsx")

# extracting tokens marked as places
tokens = ann_df.loc[ann_df["is_place"] == "yes", "capitalized_nonloc"].astype(str).tolist()

# building regex pattern from tokens
escaped_tokens = [re.escape(t) for t in tokens]
pattern = r"\b(?:" + "|".join(escaped_tokens) + r")\b"

# loading the flair ner model for ukrainian
tagger = SequenceTagger.load('lang-uk/flair-uk-ner')

# defining helper function to filter overlapping places
def filter_overlaps(places):
    sorted_places = sorted(places, key=len, reverse=True)
    filtered = []
    for place in sorted_places:
        if not any(place in longer for longer in filtered):
            filtered.append(place)
    return filtered

# defining function to extract places using flair ner and regex
def extract_places(text, tagger):
    if not isinstance(text, str):
        return ''
    sentence = Sentence(text)
    tagger.predict(sentence)
    ner_places = {e.text for e in sentence.get_spans('ner') if e.get_label('ner').value == 'LOC'}
    regex_places = set(re.findall(pattern, text))
    combined = ner_places.union(regex_places)
    final_places = filter_overlaps(combined)
    return ', '.join(sorted(final_places))

# inserting the places column immediately after TEXT_&_MEMO_VALUES
df.insert(
    df.columns.get_loc('TEXT_&_MEMO_VALUES') + 1,
    'places',
    df['TEXT_&_MEMO_VALUES'].apply(lambda x: extract_places(x, tagger))
)

# saving the updated dataframe to a new excel file
df.to_excel(
    "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/incoming_text&memo_data_for_compl_place_extraction_with_places.xlsx",
    index=False
)

# printing overall statistics for places
total = len(df)
non_empty = df['places'].astype(bool).sum()
empty = total - non_empty
print(f"total rows: {total}")
print(f"non-empty places: {non_empty} ({non_empty/total*100:.2f}%)")
print(f"empty places: {empty} ({empty/total*100:.2f}%)")

# printing average and max number of places for non-empty rows
counts = df.loc[df['places'].astype(bool), 'places'].str.count(',') + 1
print(f"average places number: {counts.mean():.2f}")
print(f"max places in one row: {counts.max()}")

# printing statistics grouped by source
for src in df['source'].unique():
    subset = df[df['source'] == src]
    total_src = len(subset)
    non_empty_src = subset['places'].astype(bool).sum()
    counts_src = subset.loc[subset['places'].astype(bool), 'places'].str.count(',') + 1
    avg_src = counts_src.mean() if not counts_src.empty else 0
    max_src = counts_src.max() if not counts_src.empty else 0
    print(f"\nsource: {src}")
    print(f" total rows: {total_src}")
    print(f" non-empty places: {non_empty_src} ({non_empty_src/total_src*100:.2f}%)")
    print(f" average places per non-empty: {avg_src:.2f}")
    print(f" max places in one row: {max_src}")

# displaying 20 random samples from the dataframe with all columns
print("\nrandom samples:")
display(df.sample(20))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/72.9M [00:00<?, ?B/s]

2025-04-23 11:09:54,776 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>
total rows: 1373
non-empty places: 653 (47.56%)
empty places: 720 (52.44%)
average places number: 1.46
max places in one row: 24

source: ru_details_mod
 total rows: 43
 non-empty places: 32 (74.42%)
 average places per non-empty: 1.94
 max places in one row: 5

source: creator
 total rows: 371
 non-empty places: 370 (99.73%)
 average places per non-empty: 1.00
 max places in one row: 1

source: Places
 total rows: 17
 non-empty places: 17 (100.00%)
 average places per non-empty: 4.18
 max places in one row: 11

source: Provenance
 total rows: 17
 non-empty places: 17 (100.00%)
 average places per non-empty: 3.00
 max places in one row: 3

source: description_02_uk
 total rows: 919
 non-empty places: 216 (23.50%)
 average places per non-empty: 1.73
 max places in one row: 3

s

,Act_ID,TEXT_&_MEMO_VALUES,places,source,incoming,ID,FILE_NAME,Sign
1161,7530.0,Інформативні звіти,,description_02_uk,14384469,3337,Incoming_14384469_meta.xlsx,4_R-3567_01_00046
851,7011.0,Тернопільська обласна комісія сприяння в робот...,Тернопільська,description_02_uk,14384469,3162,Incoming_14384469_meta.xlsx,4_R-0274_01_00068
1279,7658.0,Документи про полеглих підпільників,,description_02_uk,14384469,3465,Incoming_14384469_meta.xlsx,4_R-3567_01_00174
501,6262.0,Вишгородоцька єврейська громада Кременецького ...,"Вишгородоцька, Кременецького повіту, Лановецьк...",description_02_uk,14384469,2492,Incoming_14384469_meta.xlsx,4_0484_01_00165
792,6780.0,Відділ: житловий,,description_02_uk,14384469,2925,Incoming_14384469_meta.xlsx,4_R-0181_01_00272
38,8364.0,Документ # 1 /В Похоронное Бюро /Домоуправлени...,Сумской ул,ru_details_mod,11803151,39,Incoming_11803151_meta.xlsx,6_R-3009_27_01710
566,6554.0,Відділ: адміністративний,,description_02_uk,14384469,2699,Incoming_14384469_meta.xlsx,4_R-0181_01_00034
247,74.0,УСБУ у Львівській обл.,Львівській обл,creator,13439437,163,Incoming_13439437_meta.xlsx,1_0005_01_00075
852,6846.0,Відділ: житловий,,description_02_uk,14384469,2991,Incoming_14384469_meta.xlsx,4_R-0181_01_00338
741,6729.0,Відділ: житловий,,description_02_uk,14384469,2874,Incoming_14384469_meta.xlsx,4_R-0181_01_00221


In [ ]:
#@title EXTRACTING UNIQUE PLACES WITH ORIG_IDS, SOURCES AND VALUES

# loading the dataset from excel into a pandas dataframe
df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/incoming_text&memo_data_for_compl_place_extraction_with_places.xlsx")

# creating orig_incom&id column by concatenating incoming and ID
df['orig_incom&id'] = df['incoming'].astype(str) + "_" + df['ID'].astype(str)

# splitting places into individual entries
exploded = (
    df[['orig_incom&id', 'places', 'source', 'TEXT_&_MEMO_VALUES']]
    .dropna(subset=['places'])
    .assign(place=lambda x: x['places'].str.split(', '))
    .explode('place')
    .rename(columns={'place': 'unique_extracted_places', 'TEXT_&_MEMO_VALUES': 'text'})
    .reset_index(drop=True)
)

# aggregating orig_incom&id per place+source and capturing up to six unique example texts
grouped = (
    exploded
    .groupby(['unique_extracted_places', 'source'])
    .agg({
        'orig_incom&id': lambda ids: sorted(set(ids)),             # collect unique IDs
        'text':           lambda texts: list(dict.fromkeys(texts))[:6]  # first 6 unique samples
    })
    .reset_index()
)

# convert list of IDs into comma-separated string
grouped['orig_incom&id'] = grouped['orig_incom&id'].apply(lambda lst: ', '.join(lst))

# expanding example texts into value_1…value_6 columns
for i in range(6):
    grouped[f'value_{i+1}'] = grouped['text'].apply(lambda lst: lst[i] if len(lst) > i else '')

# dropping empty value_i columns dynamically
base_cols = ['orig_incom&id', 'unique_extracted_places', 'source']
value_cols = [col for col in grouped.columns if col.startswith('value_') and not grouped[col].eq('').all()]
result = grouped[base_cols + sorted(value_cols, key=lambda x: int(x.split('_')[1]))]

# final validation: ensure no duplicated orig_incom&id within any cell
dup_cells = []
for idx, cell in result['orig_incom&id'].items():
    ids = [i.strip() for i in str(cell).split(',')]
    if len(ids) != len(set(ids)):
        dup_cells.append((idx, cell))
if dup_cells:
    print("found duplicate orig_incom&id values in rows:")
    for idx, cell in dup_cells:
        print(f" row {idx}: {cell}")
else:
    print("all orig_incom&id values within cells are unique")

# saving the result to excel
output_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/unique_compl_extracted_places_before_cleansing.xlsx"
result.to_excel(output_path, index=False)

# printing overall statistics
total_unique_places = result['unique_extracted_places'].nunique()
print(f"total unique places: {total_unique_places}")

# counting unique places per source
source_counts = (
    result[['unique_extracted_places', 'source']]
    .drop_duplicates()
    .groupby('source')['unique_extracted_places']
    .nunique()
)
for src, count in source_counts.items():
    pct = count / total_unique_places * 100
    print(f"{src}: {count} ({pct:.2f}%) unique places")

# distribution of number of example values per row
value_count_dist = result[value_cols].notna().sum(axis=1).value_counts().sort_index()
print("\ndistribution of number of values per row:")
for num, cnt in value_count_dist.items():
    print(f"{num} values: {cnt} rows")

# top 20 most frequent places in original data
place_freq = exploded['unique_extracted_places'].value_counts().head(20)
print("\ntop 20 most frequent places:")
print(place_freq)

# distribution of number of sources per unique place
source_dist = (
    exploded[['unique_extracted_places', 'source']]
    .drop_duplicates()
    .groupby('unique_extracted_places')['source']
    .nunique()
    .value_counts()
    .sort_index()
)
print("\ndistribution of number of sources per place:")
for num, cnt in source_dist.items():
    print(f"{num} sources: {cnt} places")

# displaying 20 random rows for inspection
print("\nrandom sample of 20 rows:")
display(result.sample(20))

# processing complete message
print(f"processing complete. results saved to {output_path}")


all orig_incom&id values within cells are unique
total unique places: 158
Places: 58 (36.71%) unique places
Provenance: 10 (6.33%) unique places
creator: 12 (7.59%) unique places
description_02_uk: 11 (6.96%) unique places
note_uk: 24 (15.19%) unique places
ru_details_mod: 43 (27.22%) unique places

distribution of number of values per row:
6 values: 158 rows

top 20 most frequent places:
unique_extracted_places
Львівській обл                      294
Тернопільська                       124
Кременецького повіту                 80
Вишгородоцька                        75
Лановецького району                  75
Житомирській обл                     32
УРСР                                 17
Хмельницькій обл                     17
Украіни                              17
Харькова                             13
ТЕРНОПІЛЬСЬКОЇ ОБЛАСТІ               12
Миколаївській обл                    10
Ровенская область                     6
Рівненському району                   5
Львовская область       

,orig_incom&id,unique_extracted_places,source,value_1,value_2,value_3,value_4,value_5,value_6
131,"11803151_17, 11803151_20",Харьковскому,ru_details_mod,Документ # 1 /Харьковскому моргу /Прошу принят...,Документ # 1 /Харьковскому моргу /Домоуправлен...,,,,
119,11803151_22,Украина,ru_details_mod,Документ # 1 /Улица Тринклера 2 /1862 г. Медви...,,,,,
96,"13569102_10, 13569102_11, 13569102_12, 1356910...",Ровенская область,Places,город Ровно и Ровенский район Ровенская область,"Александрийский, Березновский, Владимирецкий, ...","Клесовский, Клеванский, Корецкий, Межеричский ...","Сарненский, Степанский и Тучинский районы Рове...","Вербовский, Демидовский, Острожецкий, Козински...",Червоноармейский район Ровенская область
108,13569102_11,Степанский,Places,"Сарненский, Степанский и Тучинский районы Рове...",,,,,
86,11803151_16,Московской улицы,ru_details_mod,Документ #1 /домоуправление № 1007 /28.XI /в б...,,,,,
132,"13439437_51, 13439437_52, 13439437_53, 1343943...",Хмельницькій обл,creator,УСБУ в Хмельницькій обл.,,,,,
138,"13439437_68, 13439437_69, 13439437_70, 1343943...",Черкаській,creator,УСБУ в Черкаській обл.,,,,,
25,13569102_9,Гощанский,Places,"Александрийский, Березновский, Владимирецкий, ...",,,,,
140,13569102_17,Черновицкий районы,Places,"города Черновицы и Сторожинец; Вижницкий, Вашк...",,,,,
135,11803151_13,Чеботарской ул,ru_details_mod,Документ # 1 /Труп гр. Симкин Шмуль Айзикович ...,,,,,


processing complete. results saved to /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/unique_compl_extracted_places_before_cleansing.xlsx


# DATA CLEANSING

In [ ]:
#@title PRIMARY CLEANSING OF EXTRACTED PLACES

# loading the unique places file into a pandas dataframe
df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/unique_compl_extracted_places_before_cleansing.xlsx")

# regex pattern for unwanted entries
deletion_pattern = r'(?:^.*(?:\d|ул|берг|кладбищ|завод|проспект|Немерзель|Степанск|Госпром).*$)'

# capturing initial row count
initial_count = len(df)

# identifying rows to delete
mask = df['unique_extracted_places'].astype(str).str.contains(deletion_pattern)
deleted_count = mask.sum()

# capturing deleted place values for reporting
deleted_places = df.loc[mask, 'unique_extracted_places'].unique()

# filtering out rows where unique_extracted_places matches the pattern
df_clean = df[~mask].reset_index(drop=True)

# printing detailed report of deleted place values
print("deleted place values:")
for place in deleted_places:
    print(place)

# saving the cleansed dataframe to a new excel file
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/unique_compl_extracted_places_after_primary_cleansing.xlsx"
df_clean.to_excel(output_file, index=False)

# printing detailed output
final_count = len(df_clean)
deleted_percent = round(deleted_count / initial_count * 100, 2)
final_percent = round(final_count / initial_count * 100, 2)

print("\nprocessing complete!")
print(f"initial rows: {initial_count} (100%)")
print(f"rows deleted: {deleted_count} ({deleted_percent}%)")
print(f"final rows: {final_count} ({final_percent}%)")

# displaying a random sample of 20 rows for inspection
print("\nrandom sample of remaining rows:")
display(df_clean.sample(20))


deleted place values:
6 ул. Воскресенская
Ветеринарной ул. № 23
Вознесенской улице
Воскресенская ул. № 6
Гольдберг
Дмитриевская ул. № 20
Дмитрова 20
Екатеринославской ул.
Карповская 17
Красно-Заводском кладбище
Ленина 7
Малой Гончаровской улице
Монастырской улице № 17
Московская ул. № 9
Московской улицы
Немерзель
Степанский
Сумской ул
Сумской улице
Тракторным заводом
Чеботарской ул
Чеботарской улице
Юмовской 9-11
доме № 4
проспекту Ленина
район Госпрома
ул. Вознесенской № 4 Гуревич
ул. Тринклера 20
ул. Шабельская № 7
№ 3
№ 7

processing complete!
initial rows: 158 (100%)
rows deleted: 31 (19.62%)
final rows: 127 (80.38%)

random sample of remaining rows:


,orig_incom&id,unique_extracted_places,source,value_1,value_2,value_3,value_4,value_5,value_6
84,14384469_2565,Романівна,note_uk,ґлибочок Малий Ободівка. Гнилиці Великі. Капус...,NaN,NaN,NaN,NaN,NaN
125,14384469_2530,Шумська,description_02_uk,Шумська єврейська громада Кременецького повіту,NaN,NaN,NaN,NaN,NaN
49,14384469_2565,Колярі,note_uk,ґлибочок Малий Ободівка. Гнилиці Великі. Капус...,NaN,NaN,NaN,NaN,NaN
54,14384469_2565,Кретівні,note_uk,ґлибочок Малий Ободівка. Гнилиці Великі. Капус...,NaN,NaN,NaN,NaN,NaN
103,11803151_41,Харьков,ru_details_mod,"Документ # 1 /Домоуправление № 1305, 13 район ...",NaN,NaN,NaN,NaN,NaN
55,"14384469_2453, 14384469_2454, 14384469_2455, 1...",Лановецького району,description_02_uk,Вишгородоцька єврейська громада Кременецького ...,NaN,NaN,NaN,NaN,NaN
21,13569102_12,Демидовский,Places,"Вербовский, Демидовский, Острожецкий, Козински...",NaN,NaN,NaN,NaN,NaN
65,14384469_2565,Максимівна,note_uk,ґлибочок Малий Ободівка. Гнилиці Великі. Капус...,NaN,NaN,NaN,NaN,NaN
43,14384469_2565,Клсбанівка,note_uk,ґлибочок Малий Ободівка. Гнилиці Великі. Капус...,NaN,NaN,NaN,NaN,NaN
6,13569102_17,Вашковский,Places,"города Черновицы и Сторожинец; Вижницкий, Вашк...",NaN,NaN,NaN,NaN,NaN


**(MANUAL LEMMATIZATION, NORMALIZATION USING TRANSLATION TO & FROM ENGLISH)**

In [ ]:
#@title REDEDUPLICATION OF EXTRACTED PLACES

# loading the manually processed unique places file
df = pd.read_excel("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/unique_compl_extracted_places_manually_processed.xlsx")

# capture initial row count
initial_count = len(df)

# drop only the unique_extracted_places column
df = df.drop(columns=['unique_extracted_places'])

# identify value columns dynamically
value_cols = [col for col in df.columns if col.startswith('value_')]

# function to aggregate and sort orig_incom&id entries
def aggregate_ids(series):
    all_ids = []
    for cell in series.dropna().astype(str):
        for part in cell.split(','):
            pid = part.strip()
            if pid:
                all_ids.append(pid)
    unique_ids = set(all_ids)
    # sort by numeric parts before and after underscore
    sorted_ids = sorted(unique_ids, key=lambda s: tuple(map(int, s.split('_'))))
    return ', '.join(sorted_ids)

# function to aggregate up to six unique example values
def aggregate_values(group):
    vals = []
    for col in value_cols:
        for cell in group[col].dropna().astype(str):
            if cell and cell not in vals:
                vals.append(cell)
    return vals[:6]

# perform grouping and aggregation, preserving other columns
grouped = df.groupby(['places_norm&lemm', 'source']).apply(
    lambda g: pd.Series({
        'orig_incom&id': aggregate_ids(g['orig_incom&id']),
        'admun_names_normalized': g['admun_names_normalized'].iloc[0],
        'admun_term': g['admun_term'].iloc[0],
        'places_norm&lemm_eng': g['places_norm&lemm_eng'].iloc[0],
        'places_norm&lemm_eng_norm': g['places_norm&lemm_eng_norm'].iloc[0],
        'admun_term_eng': g['admun_term_eng'].iloc[0],
        'note': g['note'].iloc[0],
        'values': aggregate_values(g)
    })
).reset_index()

# expand aggregated values into separate value_1…value_6 columns
for i in range(6):
    grouped[f'value_{i+1}'] = grouped['values'].apply(lambda lst: lst[i] if len(lst) > i else '')

# drop the temporary values list column
grouped = grouped.drop(columns=['values'])

# reorder columns to match original structure minus unique_extracted_places
final_cols = [
    'orig_incom&id',
    'places_norm&lemm',
    'admun_names_normalized',
    'admun_term',
    'places_norm&lemm_eng',
    'places_norm&lemm_eng_norm',
    'admun_term_eng',
    'note',
    'source'
] + [col for col in grouped.columns if col.startswith('value_')]
result = grouped[final_cols]

# capture final row count and count of merged groups
final_count = len(result)
group_sizes = df.groupby(['places_norm&lemm', 'source']).size()
merged_groups = group_sizes[group_sizes > 1]
num_merged = len(merged_groups)

# save the deduplicated result to excel
output_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/unique_compl_extracted_places_after_rededuplication.xlsx"
result.to_excel(output_path, index=False)

# print summary statistics
print(f"rows before deduplication: {initial_count}")
print(f"rows after deduplication: {final_count}")
print(f"number of merged groups: {num_merged}")

# print merged group details
print("\nmerged groups and their combined ids:")
for place, src in merged_groups.index:
    ids = result.loc[
        (result['places_norm&lemm'] == place) & (result['source'] == src),
        'orig_incom&id'
    ].iloc[0]
    print(f"{place} | {src}")

# display a random sample of deduplicated rows
print("\nrandom sample of deduplicated rows:")
display(result.sample(7))

# final message
print(f"deduplication complete. results saved to {output_path}")

rows before deduplication: 125
rows after deduplication: 115
number of merged groups: 4

merged groups and their combined ids:
Ровенская область | Places
Ровно | Places
Тернопільська область | description_02_uk
Харьков | ru_details_mod

random sample of deduplicated rows:


<ipython-input-13-b6e4f7c09aaa>:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby(['places_norm&lemm', 'source']).apply(


,orig_incom&id,places_norm&lemm,admun_names_normalized,admun_term,places_norm&lemm_eng,places_norm&lemm_eng_norm,admun_term_eng,note,source,value_1,value_2,value_3,value_4,value_5,value_6
71,13569102_12,Мизочский район,Мизоч,район,Myzoch rayon,Myzoch,rayon,NaN,Places,"Вербовский, Демидовский, Острожецкий, Козински...",,,,,
103,14384469_2565,Чагарі-Збаразькі,NaN,NaN,Chahari-Zbarazki,Chahari-Zbarazki,NaN,NaN,note_uk,ґлибочок Малий Ободівка. Гнилиці Великі. Капус...,,,,,
62,"13569102_1, 13569102_2, 13569102_3, 13569102_4",Львівський евакуаційний район,Львів,евакуаційний район,Lviv Evacuation rayon,NaN,NaN,NaN,Provenance,Головний представник уряду УРСР в справах евак...,,,,,
110,13439437_72,Чернівецька область,Черновцы,область,Chernivtsi oblast,Chernivtsi,oblast,NaN,creator,УСБУ в Чернівецькій обл.,,,,,
86,13569102_17,Садгорский район,Садгора,район,Sadgora rayon,Sadgora,rayon,NaN,Places,"города Черновицы и Сторожинец; Вижницкий, Вашк...",,,,,
76,13569102_12,Острожецкий район,Острожец,район,Ostrohets rayon,Ostrohets,rayon,NaN,Places,"Вербовский, Демидовский, Острожецкий, Козински...",,,,,
25,13569102_9,Домбровицкий район,Дубровица,район,Dubrovytsia rayon,Dubrovytsia,rayon,NaN,Places,"Александрийский, Березновский, Владимирецкий, ...",,,,,


deduplication complete. results saved to /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/unique_compl_extracted_places_after_rededuplication.xlsx


# MATCHING WITH SAPIR

In [ ]:
#@title CREATING A TRANSLITERATION DICTIONARY FOR REGEX-BASED MATCHING

# defining base transliteration dictionary
base_dict = {
    'а': ['a'], 'б': ['b'], 'в': ['v','w'], 'г': ['h','g'], 'ґ': ['g'],
    'д': ['d','j'], 'е': ['e','ye','ie'], 'є': ['ye','je','ie'],
    'ж': ['zh','j','rz'], 'з': ['z'], 'и': ['y','i'], 'і': ['i'],
    'ї': ['yi','ji','ii','i','e','ye','ie','je'], 'й': ['y','j','i'],
    'к': ['k','c','q'], 'л': ['l'], 'м': ['m'], 'н': ['n'], 'о': ['o'],
    'п': ['p'], 'р': ['r'], 'с': ['s'], 'т': ['t'], 'у': ['u','ou'],
    'ф': ['f','ph'], 'х': ['kh','ch','h'], 'ц': ['ts','tz','z','c'],
    'ч': ['ch','tch','tsch','cz'], 'ш': ['sh','sch','sz'], 'щ': ['shch','sch','sc','szcz'],
    'ю': ['yu','ju','iu'], 'я': ['ya','ja','ia']
}

# defining extended transliteration dictionary
extended_dict = {
    'а': ['a'], 'б': ['b'], 'в': ['v','w','l'], 'г': ['h','g'], 'ґ': ['g'],
    'д': ['d','j'], 'е': ['e','ye','ie'], 'є': ['ye','je','ie','e'],
    'ж': ['zh','j','rz'], 'з': ['z'], 'и': ['y','i'], 'і': ['i','o','y','e'],
    'ї': ['yi','ji','ii','i','e','ye','ie','je'], 'й': ['y','j','i'],
    'к': ['k','c','q'], 'л': ['l','ł'], 'м': ['m'], 'н': ['n'], 'о': ['o'],
    'п': ['p'], 'р': ['r','rz'], 'с': ['s'], 'т': ['t'], 'у': ['u','ou','ó'],
    'ф': ['f','ph'], 'х': ['kh','ch','h'], 'ц': ['ts','tz','z','c'],
    'ч': ['ch','tch','tsch','cz'], 'ш': ['sh','sch','sz'], 'щ': ['shch','sch','sc','szcz'],
    'ю': ['yu','ju','iu'], 'я': ['ya','ja','ia','a']
}

# defining combined mapping object
mapping = {
    "base_dict": base_dict,
    "extended_dict": extended_dict
}

# saving mapping to json file
with open("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/translit_dicts.json", "w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

# printing confirmation message
print("mapping dicts saved to /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/translit_dicts.json")


mapping dicts saved to /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/translit_dicts.json


In [ ]:
#@title PLACE MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# loading transliteration dictionaries from json
with open("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/translit_dicts.json", "r", encoding="utf-8") as f:
    dicts = json.load(f)
base_dict = dicts["base_dict"]
extended_dict = dicts["extended_dict"]

# defining similarity threshold for fuzzy matching
similarity_threshold = 85

# defining function to build regex pattern from transliteration dictionary
def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alts = ext_dict[char]
            pattern += "(" + "|".join(re.escape(a) for a in alts) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# setting file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/final_compl_places_with_mach_transl_&_sapir_category.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# normalizing text columns
df_final["final_places_eng_norm"] = df_final["final_places_eng"].astype(str).str.lower().str.strip()
df_sapir["place_norm"] = df_sapir["place"].astype(str).str.lower().str.strip()
df_final_place = df_final[df_final["sapir_category"].astype(str).str.lower() == "place"].copy()

# precomputing regex expressions
df_final_place["final_places_regex"] = df_final_place["final_places"].str.lower().map(
    lambda x: build_regex_pattern(str(x), extended_dict)
)

output_rows = []
match_counts = {}
unique_regex_matches = set()
unique_fuzzy_matches = set()

# iterating over final places
for _, row in df_final_place.iterrows():
    query_regex = row["final_places_regex"]
    query_eng = row["final_places_eng_norm"]
    source_first = row["final_places"][0].lower() if row["final_places"] else ""

    # performing base filtering
    filtered_base = [
        idx for idx, sr in df_sapir.iterrows()
        if source_first in base_dict and sr["place_norm"] and sr["place_norm"][0] in base_dict[source_first]
    ]

    # performing regex matching
    filtered_regex = [
        idx for idx in filtered_base
        if re.fullmatch(query_regex, df_sapir.at[idx, "place_norm"])
    ]

    if filtered_regex:
        final_cands = filtered_regex
        method = "regex"
    else:
        # performing fuzzy matching
        choices = df_sapir["place_norm"].tolist()
        matches = process.extractBests(
            query_eng, choices,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=similarity_threshold,
            limit=len(choices)
        )
        final_cands = []
        for m in matches:
            if len(m) == 3:
                final_cands.append(m[2])
            else:
                final_cands.extend([i for i, v in enumerate(choices) if v == m[0]])
        method = "fuzzy"

    # performing keyword filtering
    if len(final_cands) > 1:
        kws = ["Ukraine", "Poland"]
        kw_cands = [
            i for i in final_cands
            if any(kw in df_sapir.at[i, "TIENG"] for kw in kws)
        ]
        if kw_cands:
            final_cands = kw_cands

    # counting matches
    match_counts[row["final_places_eng"]] = len(final_cands)
    val = row["final_places_eng"]
    if final_cands:
        if method == "regex":
            unique_regex_matches.add(val)
            unique_fuzzy_matches.discard(val)
        else:
            if val not in unique_regex_matches and val not in unique_fuzzy_matches:
                unique_fuzzy_matches.add(val)

    # handling no-match cases
    if not final_cands:
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "PLACE_STRING_CYR": row["final_places"],
            "PLACE_STRING_ENG": row["final_places_eng"],
            "SAPIR_PLACE_MATCH": "",
            "SAPIR_PLACE_ID": "",
            "SAPIR_PLACE_VALUE": "",
            "COUNT_MATCHES": 0
        })

    # recording matched rows
    for idx in final_cands:
        sr = df_sapir.iloc[idx]
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "PLACE_STRING_CYR": row["final_places"],
            "PLACE_STRING_ENG": row["final_places_eng"],
            "SAPIR_PLACE_MATCH": sr["place"],
            "SAPIR_PLACE_ID": sr["book_id"],
            "SAPIR_PLACE_VALUE": sr["TIENG"],
            "COUNT_MATCHES": len(final_cands)
        })

# building and saving output dataframe
df_output = pd.DataFrame(output_rows)
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_sapir_regex_&_fuzzy_match_place.xlsx"
df_output.to_excel(output_file, index=False)

# summarizing match results
total = df_final_place["final_places_eng"].nunique()
with_matches = sum(1 for c in match_counts.values() if c > 0)
no_matches = total - with_matches
dist = {}
for c in match_counts.values():
    dist[c] = dist.get(c, 0) + 1
report = [
    f"total unique final_places_eng values processed: {total}",
    f"values with one or more matches: {with_matches} ({with_matches/total*100:.2f}%)",
    f"values with no matches: {no_matches} ({no_matches/total*100:.2f}%)"
]
for c in sorted(dist):
    report.append(
        f"values with exactly {c} match(es): {dist[c]} ({dist[c]/total*100:.2f}%)"
    )
display(Markdown("**summary report:**"))
display(Markdown("\n\n".join(report)))

# computing unique match statistics
unique_total = len(unique_regex_matches | unique_fuzzy_matches)
r_pct = (len(unique_regex_matches) / unique_total * 100) if unique_total else 0
f_pct = (len(unique_fuzzy_matches) / unique_total * 100) if unique_total else 0
display(Markdown("**unique match method statistics (by final_places_eng):**"))
display(Markdown(f"unique regex matches: {len(unique_regex_matches)} ({r_pct:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({f_pct:.2f}%)"))

# displaying random matched samples
df_matches = df_output[df_output["COUNT_MATCHES"] > 0].drop_duplicates(
    subset=["PLACE_STRING_CYR", "PLACE_STRING_ENG"]
)
sample_df = df_matches.sample(n=10, random_state=42) if len(df_matches) >= 10 else df_matches
display(Markdown("**10 random sample rows with matches:**"))
display(sample_df[[
    "PLACE_STRING_CYR", "PLACE_STRING_ENG",
    "SAPIR_PLACE_MATCH", "SAPIR_PLACE_ID",
    "SAPIR_PLACE_VALUE", "COUNT_MATCHES"
]])



**loading excel files...**

**summary report:**

total unique final_places_eng values processed: 36

values with one or more matches: 32 (88.89%)

values with no matches: 4 (11.11%)

values with exactly 0 match(es): 4 (11.11%)

values with exactly 1 match(es): 10 (27.78%)

values with exactly 2 match(es): 8 (22.22%)

values with exactly 3 match(es): 1 (2.78%)

values with exactly 4 match(es): 4 (11.11%)

values with exactly 5 match(es): 3 (8.33%)

values with exactly 6 match(es): 2 (5.56%)

values with exactly 7 match(es): 1 (2.78%)

values with exactly 8 match(es): 1 (2.78%)

values with exactly 11 match(es): 1 (2.78%)

values with exactly 29 match(es): 1 (2.78%)

**unique match method statistics (by final_places_eng):**

unique regex matches: 25 (78.12%)

unique fuzzy matches: 7 (21.88%)

**10 random sample rows with matches:**

,PLACE_STRING_CYR,PLACE_STRING_ENG,SAPIR_PLACE_MATCH,SAPIR_PLACE_ID,SAPIR_PLACE_VALUE,COUNT_MATCHES
122,Харьков,Kharkiv,Kharkov,5426482,"Kharkov,Kharkov City,Kharkov,Ukraine (USSR)",2
42,Колярі,Kolyari,Kolare,5462102,"Kolare,Hont,Slovakia,Czechoslovakia",1
77,Романівка,Romanivka,Romanovka,10094742,"Romanovka,Chemerovtsy,Kamenets Podolsk,Ukraine...",29
44,Кретівці,Kretivtsi,Kretowce,5729251,"Kretowce,Zbaraz,Tarnopol,Poland",1
24,Зарубинці,Zarubyntsi,Zarubince,5443270,"Zarubince,Zbaraz,Tarnopol,Poland",6
30,Золочев,Zolochiv,Zolochev,5760249,"Zolochev,Zolochev,Kharkov,Ukraine (USSR)",2
125,Черновцы,Chernivtsi,Chernevtsi,5479300,"Chernevtsi,Chernevtsy,Vinnitsa,Ukraine (USSR)",2
106,Рохманів,Rokhmaniv,Romaniv,5533228,"Romaniv,Bobrka,Lwow,Poland",4
35,Клебанівка,Klebanivka,Klebanowka,5463668,"Klebanowka,Zbaraz,Tarnopol,Poland",1
0,Івачів,Ivachiv,Ivachiv,13884785,"Ivachiv,Zborow,Tarnopol,Poland",2


In [ ]:
#@title UNPIVOTED PLACES WITH SAPIR

import os
import pandas as pd
from IPython.display import display, Markdown

# defining directories and filenames
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"
mapping_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"

manual_file = "compl_places_sapir_regex_&_fuzzy_match_place_with_man.xlsx"
mapping_file = "t_General_Act_IDs_with_old_ID.xlsx"

# loading dataframes
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(mapping_dir, mapping_file))

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# converting mapping id fields to string uniformly
df_mapping["Comb_ID"] = df_mapping["Comb_ID"].apply(normalize_id).astype("string")
df_mapping["Act_ID"]  = df_mapping["Act_ID"].apply(normalize_id).astype("string")
df_mapping["Sign"]    = df_mapping["Sign"].apply(normalize_id).astype("string")

# filtering out removed rows
df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

# preparing new columns for output
df_manual["source"] = df_manual["source"]
df_manual["place_cyr"] = df_manual["PLACE_STRING_CYR"]
df_manual["place_sapir"] = df_manual["SAPIR_PLACE_MATCH"].where(df_manual["STATUS"] == "to use", "")
df_manual["place_missing"] = df_manual["PLACE_STRING_ENG"].where(df_manual["STATUS"] == "to export", "")
df_manual["full_sapir"] = df_manual["SAPIR_PLACE_VALUE"]

# splitting and exploding comb_id lists
df_manual["comb_id_list"] = df_manual["orig_incom&id"].astype(str).apply(
    lambda s: [normalize_id(item) for item in s.split(",") if item.strip()]
)
df_exploded = df_manual.explode("comb_id_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"comb_id_list": "comb_id"})
df_exploded["comb_id"] = df_exploded["comb_id"].astype("string")

# merging with mapping on comb_id
df_merged = pd.merge(
    df_exploded,
    df_mapping[["Comb_ID", "Act_ID", "Sign"]],
    left_on="comb_id",
    right_on="Comb_ID",
    how="left"
)

# filling missing Act_ID/Sign with placeholder
df_merged["Act_ID"] = df_merged["Act_ID"].fillna("missing")
df_merged["Sign"]   = df_merged["Sign"].fillna("missing")

# preparing final unpivoted output
df_merged["sapir_id"] = df_merged["SAPIR_PLACE_ID"].apply(normalize_id) \
    .where(df_merged["STATUS"] == "to use", "") \
    .astype("string")
df_merged["full_sapir"] = df_merged["full_sapir"].where(df_merged["STATUS"] == "to use", "")

cols_output = [
    "source", "place_cyr", "place_sapir", "place_missing",
    "comb_id", "Act_ID", "Sign", "sapir_id", "full_sapir", "MISSING_GPS"
]
df_output = df_merged[cols_output].copy()

# preparing missing sapir places output
df_missing = df_merged[df_merged["STATUS"] == "to export"][[
    "comb_id", "source", "Act_ID", "Sign",
    "PLACE_STRING_CYR", "PLACE_STRING_ENG", "MISSING_GPS"
]].drop_duplicates(subset=["comb_id", "PLACE_STRING_CYR"])
df_missing.to_excel(os.path.join(file_dir, "compl_places_missing_in_sapir_places.xlsx"), index=False)

# computing statistics
total_manual = len(df_manual)
total_exploded = len(df_output)
perc_explosion = total_exploded / total_manual * 100 if total_manual else 0
source_counts = df_output["source"].value_counts()
source_perc = source_counts / source_counts.sum() * 100
unique_places = df_output["place_cyr"].nunique()
unique_by_src = df_output.groupby("source")["place_cyr"].nunique()
status_counts = df_manual["STATUS"].value_counts()
missing_map_count = df_output[(df_output["Act_ID"] == "missing") & (df_output["Sign"] == "missing")].shape[0]
missing_map_pct = missing_map_count / total_exploded * 100 if total_exploded else 0
total_missing = len(df_missing)
gps_dashes = df_missing["MISSING_GPS"].astype(str).str.contains("---").sum()
perc_gps_dashes = gps_dashes / total_missing * 100 if total_missing else 0

# building summary
source_lines = "\n".join(f"- {src}: {cnt} ({source_perc[src]:.2f}%)"
                         for src, cnt in source_counts.items())
unique_lines = "\n".join(f"- {src}: {cnt}"
                         for src, cnt in unique_by_src.items())
status_line = ", ".join(f"{st}: {cnt}"
                         for st, cnt in status_counts.items())

summary_md = f"""
## summary report

- **total manual rows:** {total_manual}
- **total exploded rows:** {total_exploded} ({perc_explosion:.2f}%)

### source breakdown
{source_lines}

- **unique place_cyr overall:** {unique_places}

### unique place_cyr by source
{unique_lines}

- **manual status counts:** {status_line}

- **mapping missing comb_ids:** {missing_map_count} ({missing_map_pct:.2f}%)
- **missing sapir places rows:** {total_missing}, dashes in GPS: {gps_dashes} ({perc_gps_dashes:.2f}%)
"""

# displaying summary in markdown
display(Markdown(summary_md))

# saving unpivoted output
df_output.to_excel(os.path.join(file_dir, "compl_places_unpivoted_with_sapir_places.xlsx"), index=False)

# showing samples
display(Markdown("### sample of 50 unpivoted rows"))
display(df_output.sample(n=50, random_state=42) if total_exploded >= 50 else df_output)

display(Markdown("### sample of 15 missing rows"))
display(df_missing.sample(n=15, random_state=42) if total_missing >= 15 else df_missing)



## summary report

- **total manual rows:** 49
- **total exploded rows:** 142 (289.80%)

### source breakdown
- description_02_uk: 84 (59.15%)
- note_uk: 34 (23.94%)
- ru_details_mod: 18 (12.68%)
- Places: 6 (4.23%)

- **unique place_cyr overall:** 36

### unique place_cyr by source
- Places: 5
- description_02_uk: 6
- note_uk: 24
- ru_details_mod: 1

- **manual status counts:** to use: 28, to export: 21

- **mapping missing comb_ids:** 3 (2.11%)
- **missing sapir places rows:** 9, dashes in GPS: 1 (11.11%)


### sample of 50 unpivoted rows

,source,place_cyr,place_sapir,place_missing,comb_id,Act_ID,Sign,sapir_id,full_sapir,MISSING_GPS
131,note_uk,Кобилля,,Kobylya,14384469_2565,6335,4_0484_01_00239,,,"49.77616714368964, 25.608979294133956"
19,note_uk,Капустинці,,Kapustyntsi,14384469_2565,6335,4_0484_01_00239,,,"49.727949869193885, 25.81553084109276"
81,description_02_uk,Вишгородок,Vyshgorodok,,14384469_2490,6260,4_0484_01_00163,5558074,"Vyshgorodok,Krzemieniec,Wolyn,Poland",NaN
96,description_02_uk,Вишгородок,Vyshgorodok,,14384469_2505,6275,4_0484_01_00178,5558074,"Vyshgorodok,Krzemieniec,Wolyn,Poland",NaN
56,description_02_uk,Вишгородок,Vyshgorodok,,14384469_2465,6235,4_0484_01_00138,5558074,"Vyshgorodok,Krzemieniec,Wolyn,Poland",NaN
12,note_uk,Колярі,,Kolyari,14384469_2565,6335,4_0484_01_00239,,,---
98,description_02_uk,Вишгородок,Vyshgorodok,,14384469_2507,6277,4_0484_01_00180,5558074,"Vyshgorodok,Krzemieniec,Wolyn,Poland",NaN
65,description_02_uk,Вишгородок,Vyshgorodok,,14384469_2474,6244,4_0484_01_00147,5558074,"Vyshgorodok,Krzemieniec,Wolyn,Poland",NaN
66,description_02_uk,Вишгородок,Vyshgorodok,,14384469_2475,6245,4_0484_01_00148,5558074,"Vyshgorodok,Krzemieniec,Wolyn,Poland",NaN
18,note_uk,Капустинці,,Kapustyntsi,14384469_2565,6335,4_0484_01_00239,,,"49.727949869193885, 25.81553084109276"


### sample of 15 missing rows

,comb_id,source,Act_ID,Sign,PLACE_STRING_CYR,PLACE_STRING_ENG,MISSING_GPS
1,14384469_2565,note_uk,6335,4_0484_01_00239,Малий Глибочок,Malyi Hlybochok,"49.700847637596894, 25.805466093132214"
2,14384469_2565,note_uk,6335,4_0484_01_00239,Чагарі-Збаразькі,Chahari-Zbarazki,"49.61652116370083, 25.91255337411782"
3,14384469_2565,note_uk,6335,4_0484_01_00239,Шили,Shyli,"49.69359128958817, 25.984773613503194"
12,14384469_2565,note_uk,6335,4_0484_01_00239,Колярі,Kolyari,---
18,14384469_2565,note_uk,6335,4_0484_01_00239,Капустинці,Kapustyntsi,"49.727949869193885, 25.81553084109276"
21,14384469_2565,note_uk,6335,4_0484_01_00239,Сухівці,Sukhivtsi,"49.66182244369368, 26.04608431603258"
119,14384469_2565,note_uk,6335,4_0484_01_00239,Ободівка,Obodivka,"49.63030961659456, 26.0224883430252"
125,14384469_2529,description_02_uk,6299,4_0484_01_00202,Рохманів,Rokhmaniv,"50.12908826346198, 26.084423856220564"
129,14384469_2565,note_uk,6335,4_0484_01_00239,Кобилля,Kobylya,"49.77616714368964, 25.608979294133956"


In [ ]:
#@title DISTRICT MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# loading transliteration dictionaries from json
with open("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/translit_dicts.json", "r", encoding="utf-8") as f:
    dicts = json.load(f)
base_dict = dicts["base_dict"]
extended_dict = dicts["extended_dict"]

# defining similarity threshold for fuzzy matching
similarity_threshold = 85

# defining function to build regex pattern from transliteration dictionary
def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alts = ext_dict[char]
            pattern += "(" + "|".join(re.escape(a) for a in alts) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# setting file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/final_compl_places_with_mach_transl_&_sapir_category.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# filtering sapir table to include only rows where place == "<>"
df_sapir = df_sapir[df_sapir["place"] == "<>"]
df_sapir = df_sapir.reset_index(drop=True)

# normalizing text columns
df_final["final_places_eng_norm"] = df_final["norm_admun_name_eng"].astype(str).str.lower().str.strip()
df_sapir["district_norm"] = df_sapir["district"].astype(str).str.lower().str.strip()
df_final_district = df_final[df_final["sapir_category"].astype(str).str.lower() == "district"].copy()

# precomputing regex expressions
df_final_district["final_places_regex"] = df_final_district["norm_admun_name_cyr"].str.lower().map(
    lambda x: build_regex_pattern(str(x), extended_dict)
)

output_rows = []
match_counts = {}
unique_regex_matches = set()
unique_fuzzy_matches = set()

# iterating over final districts
for _, row in df_final_district.iterrows():
    query_regex = row["final_places_regex"]
    query_eng = row["final_places_eng_norm"]
    source_first = row["norm_admun_name_cyr"][0].lower() if row["norm_admun_name_cyr"] else ""

    # performing base filtering
    filtered_base = [
        idx for idx, sr in df_sapir.iterrows()
        if source_first in base_dict and sr["district_norm"] and sr["district_norm"][0] in base_dict[source_first]
    ]

    # performing regex matching
    filtered_regex = [
        idx for idx in filtered_base
        if re.fullmatch(query_regex, df_sapir.at[idx, "district_norm"])
    ]

    if filtered_regex:
        final_cands = filtered_regex
        method = "regex"
    else:
        # performing fuzzy matching
        choices = df_sapir["district_norm"].tolist()
        matches = process.extractBests(
            query_eng, choices,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=similarity_threshold,
            limit=len(choices)
        )
        final_cands = []
        for m in matches:
            if len(m) == 3:
                final_cands.append(m[2])
            else:
                final_cands.extend([i for i, v in enumerate(choices) if v == m[0]])
        method = "fuzzy"

    # performing keyword filtering
    if len(final_cands) > 1:
        kws = ["Ukraine", "Poland"]
        kw_cands = [
            i for i in final_cands
            if any(kw in df_sapir.at[i, "TIENG"] for kw in kws)
        ]
        if kw_cands:
            final_cands = kw_cands

    # counting matches
    match_counts[row["final_places_eng"]] = len(final_cands)
    val = row["final_places_eng"]
    if final_cands:
        if method == "regex":
            unique_regex_matches.add(val)
            unique_fuzzy_matches.discard(val)
        else:
            if val not in unique_regex_matches and val not in unique_fuzzy_matches:
                unique_fuzzy_matches.add(val)

    # handling no-match cases
    if not final_cands:
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "DISTRICT_STRING_CYR": row["final_places"],
            "DISTRICT_STRING_ENG": row["final_places_eng"],
            "SAPIR_DISTRICT_MATCH": "",
            "SAPIR_DISTRICT_ID": "",
            "SAPIR_DISTRICT_VALUE": "",
            "COUNT_MATCHES": 0
        })

    # recording matched rows
    for idx in final_cands:
        sr = df_sapir.iloc[idx]
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "DISTRICT_STRING_CYR": row["final_places"],
            "DISTRICT_STRING_ENG": row["final_places_eng"],
            "SAPIR_DISTRICT_MATCH": sr["district"],
            "SAPIR_DISTRICT_ID": sr["book_id"],
            "SAPIR_DISTRICT_VALUE": sr["TIENG"],
            "COUNT_MATCHES": len(final_cands)
        })

# building and saving output dataframe
df_output = pd.DataFrame(output_rows)
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_sapir_regex_&_fuzzy_match_district.xlsx"
df_output.to_excel(output_file, index=False)

# summarizing match results
total = df_final_district["final_places_eng"].nunique()
with_matches = sum(1 for c in match_counts.values() if c > 0)
no_matches = total - with_matches
dist = {}
for c in match_counts.values():
    dist[c] = dist.get(c, 0) + 1
report = [
    f"total unique final_places_eng values processed: {total}",
    f"values with one or more matches: {with_matches} ({with_matches/total*100:.2f}%)",
    f"values with no matches: {no_matches} ({no_matches/total*100:.2f}%)"
]
for c in sorted(dist):
    report.append(
        f"values with exactly {c} match(es): {dist[c]} ({dist[c]/total*100:.2f}%)"
    )
display(Markdown("**summary report:**"))
display(Markdown("\n\n".join(report)))

# computing unique match statistics
unique_total = len(unique_regex_matches | unique_fuzzy_matches)
r_pct = (len(unique_regex_matches) / unique_total * 100) if unique_total else 0
f_pct = (len(unique_fuzzy_matches) / unique_total * 100) if unique_total else 0
display(Markdown("**unique match method statistics (by final_places_eng):**"))
display(Markdown(f"unique regex matches: {len(unique_regex_matches)} ({r_pct:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({f_pct:.2f}%)"))

# displaying random matched samples
df_matches = df_output[df_output["COUNT_MATCHES"] > 0].drop_duplicates(
    subset=["DISTRICT_STRING_CYR", "DISTRICT_STRING_ENG"]
)
sample_df = df_matches.sample(n=10, random_state=42) if len(df_matches) >= 10 else df_matches
display(Markdown("**10 random sample rows with matches:**"))
display(sample_df[
    [
        "DISTRICT_STRING_CYR",
        "DISTRICT_STRING_ENG",
        "SAPIR_DISTRICT_MATCH",
        "SAPIR_DISTRICT_ID",
        "SAPIR_DISTRICT_VALUE",
        "COUNT_MATCHES"
    ]
])


**loading excel files...**

**summary report:**

total unique final_places_eng values processed: 52

values with one or more matches: 13 (25.00%)

values with no matches: 39 (75.00%)

values with exactly 0 match(es): 39 (75.00%)

values with exactly 1 match(es): 11 (21.15%)

values with exactly 2 match(es): 1 (1.92%)

values with exactly 4 match(es): 1 (1.92%)

**unique match method statistics (by final_places_eng):**

unique regex matches: 6 (46.15%)

unique fuzzy matches: 7 (53.85%)

**10 random sample rows with matches:**

,DISTRICT_STRING_CYR,DISTRICT_STRING_ENG,SAPIR_DISTRICT_MATCH,SAPIR_DISTRICT_ID,SAPIR_DISTRICT_VALUE,COUNT_MATCHES
59,Червоноармейский район,Chervonoarmiisk rayon,Chervonoarmeysk,6183474,"<>,Chervonoarmeysk,Zhitomir,Ukraine (USSR)",1
43,Любомльский район,Liuboml rayon,Luboml,7531488,"<>,Luboml,Wolyn,Poland",1
0,Александрийский район,Oleksandriya rayon,Aleksandriya,7560135,"<>,Aleksandriya,Kirovograd,Ukraine (USSR)",1
42,Любашевский район,Liubashivka rayon,Lyubashevka,5714651,"<>,Lyubashevka,Odessa,Ukraine (USSR)",1
21,Здолбуновский район,Zdolbuniv rayon,Zdolbunow,9322908,"<>,Zdolbunow,Wolyn,Poland",1
9,Глубоковский район,Hlybokyi rayon,Glubokiy,9524337,"<>,Glubokiy,Rostov,Russia (USSR)",1
1,Березновский район,Berezne rayon,Berezna,7550651,"<>,Berezna,Chernigov,Ukraine (USSR)",2
62,Шацкий район,Shatsk rayon,Shatsk,6627018,"<>,Shatsk,Ryazan,Russia (USSR)",1
17,Дубнівський район,Dubno rayon,Dubno,8419785,"<>,Dubno,Wolyn,Poland",1
33,Луцкий район,Lutsk rayon,Slutsk,6506656,"<>,Slutsk,Minsk,Belorussia (USSR)",4


In [ ]:
#@title UNPIVOTED DISTRICTS WITH SAPIR

# defining directories and filenames
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"
mapping_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"

manual_file = "compl_places_sapir_regex_&_fuzzy_match_district_with_man.xlsx"
mapping_file = "t_General_Act_IDs_with_old_ID.xlsx"

# loading dataframes
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(mapping_dir, mapping_file))

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# converting mapping id fields to string uniformly
df_mapping["Comb_ID"] = df_mapping["Comb_ID"].apply(normalize_id).astype("string")
df_mapping["Act_ID"]  = df_mapping["Act_ID"].apply(normalize_id).astype("string")
df_mapping["Sign"]    = df_mapping["Sign"].apply(normalize_id).astype("string")

# filtering out removed rows
df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

# preparing new columns for output
df_manual["source"]                        = df_manual["source"]
df_manual["district_cyr"]                  = df_manual["ORIG_DISTRICT_STRING_CYR"]
df_manual["norm_district_name_cyr"]        = df_manual["NORM_DISTRICT_NAME_CYR"]
df_manual["norm_district_type_cyr"]        = df_manual["NORM_DISTRICT_TYPE_CYR"]
df_manual["district_sapir"]                = df_manual["SAPIR_DISTRICT_MATCH"].where(df_manual["STATUS"] == "to use", "")
df_manual["district_missing"]              = df_manual["NORM_DISTRICT_NAME_ENG"].where(df_manual["STATUS"] == "to export", "")
df_manual["full_sapir"]                    = df_manual["SAPIR_DISTRICT_VALUE"].where(df_manual["STATUS"] == "to use", "")
df_manual["parachronistic_district_sapir"] = df_manual["parachronistic_district_sapir"]
df_manual["parachronistic_id_sapir"] = df_manual["parachronistic_id_sapir"].apply(normalize_id).astype("string")

# splitting and exploding comb_id lists
df_manual["comb_id_list"] = df_manual["orig_incom&id"].astype(str).apply(
    lambda s: [normalize_id(item) for item in s.split(",") if item.strip()]
)
df_exploded = df_manual.explode("comb_id_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"comb_id_list": "comb_id"})
df_exploded["comb_id"] = df_exploded["comb_id"].astype("string")

# merging with mapping on comb_id
df_merged = pd.merge(
    df_exploded,
    df_mapping[["Comb_ID", "Act_ID", "Sign"]],
    left_on="comb_id",
    right_on="Comb_ID",
    how="left"
)

# filling missing Act_ID/Sign with placeholder
df_merged["Act_ID"] = df_merged["Act_ID"].fillna("missing").astype("string")
df_merged["Sign"]   = df_merged["Sign"].fillna("missing").astype("string")

# normalizing sapir_id and filtering by STATUS
df_merged["sapir_id"] = df_merged["SAPIR_DISTRICT_ID"].apply(normalize_id).astype("string")
df_merged["sapir_id"] = df_merged["sapir_id"].where(df_merged["STATUS"] == "to use", "")
df_merged["full_sapir"] = df_merged["full_sapir"].where(df_merged["STATUS"] == "to use", "")

# preparing final unpivoted output
cols_output = [
    "source",
    "district_cyr",
    "norm_district_name_cyr",
    "norm_district_type_cyr",
    "district_sapir",
    "district_missing",
    "parachronistic_district_sapir",
    "parachronistic_id_sapir",
    "comb_id",
    "Act_ID",
    "Sign",
    "sapir_id",
    "full_sapir",
    "MISS_CENTER_GPS",
    "MISS_REF1"
]
df_output = df_merged[cols_output].copy()

# preparing missing sapir districts output by STATUS == "to export"
df_missing = df_merged[df_merged["STATUS"] == "to export"][
    ["comb_id",
     "Act_ID",
     "Sign",
     "source",
     "ORIG_DISTRICT_STRING_CYR",
     "NORM_DISTRICT_NAME_CYR",
     "NORM_DISTRICT_TYPE_CYR",
     "NORM_DISTRICT_NAME_ENG",
     "NORM_DISTRICT_TYPE_ENG",
     "MISS_CENTER_GPS",
     "MISS_REF1",
     "parachronistic_district_sapir",
     "parachronistic_id_sapir"]
].drop_duplicates(subset=["comb_id", "ORIG_DISTRICT_STRING_CYR"])

# computing statistics
total_manual    = len(df_manual)
total_exploded  = len(df_output)
perc_explosion  = total_exploded / total_manual * 100 if total_manual else 0
source_counts   = df_output["source"].value_counts()
source_perc     = source_counts / source_counts.sum() * 100
unique_district = df_output["district_cyr"].nunique()
unique_by_src   = df_output.groupby("source")["district_cyr"].nunique()
status_counts   = df_manual["STATUS"].value_counts()
missing_map_count = df_output[(df_output["Act_ID"] == "missing") & (df_output["Sign"] == "missing")].shape[0]
missing_map_pct   = missing_map_count / total_exploded * 100 if total_exploded else 0
total_missing      = len(df_missing)
gps_dashes        = df_missing["MISS_CENTER_GPS"].astype(str).str.contains("---").sum()
perc_gps_dashes   = gps_dashes / total_missing * 100 if total_missing else 0

# building summary
source_lines = "\n".join(f"- {src}: {cnt} ({source_perc[src]:.2f}%)"
                         for src, cnt in source_counts.items())
unique_lines = "\n".join(f"- {src}: {cnt}"
                         for src, cnt in unique_by_src.items())
status_line  = ", ".join(f"{st}: {cnt}"
                         for st, cnt in status_counts.items())

summary_md = f"""
## summary report

- **total manual rows:** {total_manual}
- **total exploded rows:** {total_exploded} ({perc_explosion:.2f}%)

### source breakdown
{source_lines}

- **unique district_cyr overall:** {unique_district}

### unique district_cyr by source
{unique_lines}

- **manual status counts:** {status_line}

- **mapping missing comb_ids:** {missing_map_count} ({missing_map_pct:.2f}%)
- **missing sapir districts rows:** {total_missing}, dashes in GPS: {gps_dashes} ({perc_gps_dashes:.2f}%)
"""

# displaying summary in markdown
display(Markdown(summary_md))

# saving outputs
df_output.to_excel(os.path.join(file_dir, "compl_places_unpivoted_with_sapir_district.xlsx"), index=False)
df_missing.to_excel(os.path.join(file_dir, "compl_places_missing_in_sapir_district.xlsx"), index=False)

# showing sample sections
display(Markdown("### sample of 20 unpivoted rows"))
display(df_output.sample(n=20, random_state=42) if total_exploded >= 20 else df_output)
display(Markdown("### sample of 20 missing rows"))
display(df_missing.sample(n=20, random_state=42) if total_missing >= 20 else df_missing)



## summary report

- **total manual rows:** 63
- **total exploded rows:** 225 (357.14%)

### source breakdown
- description_02_uk: 157 (69.78%)
- Places: 52 (23.11%)
- Provenance: 16 (7.11%)

- **unique district_cyr overall:** 53

### unique district_cyr by source
- Places: 46
- Provenance: 7
- description_02_uk: 3

- **manual status counts:** to export: 61, to use: 2

- **mapping missing comb_ids:** 0 (0.00%)
- **missing sapir districts rows:** 134, dashes in GPS: 1 (0.75%)


### sample of 20 unpivoted rows

,source,district_cyr,norm_district_name_cyr,norm_district_type_cyr,district_sapir,district_missing,parachronistic_district_sapir,parachronistic_id_sapir,comb_id,Act_ID,Sign,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1
9,Places,Гощанский район,Гоща,район,,Hoshcha,NaN,NaN,13569102_9,1506,3_3229_48_00015,,,"50.60062795521478, 26.678693320354636",https://uk.wikipedia.org/wiki/%D0%93%D0%BE%D1%...
184,Places,Мацеевский район,Мацеев,район,,Matseiv,NaN,NaN,13569102_15,1512,3_3229_50_00007,,,"51.23023602527496, 24.34322021164085",https://uk.wikipedia.org/wiki/%D0%9C%D0%B0%D1%...
120,description_02_uk,Лановецький район,Ланівці,район,,Lanivtsi,NaN,NaN,14384469_2467,6237,4_0484_01_00140,,,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...
207,Provenance,Дубнівський район,Дубно,район,,Dubno,"<>,Dubno,Wolyn,Poland",8419785.0,13569102_13,1509,3_3229_49_00009,,,"50.40094498982122, 25.76005320821866",https://uk.wikipedia.org/wiki/%D0%94%D1%83%D0%...
148,description_02_uk,Лановецький район,Ланівці,район,,Lanivtsi,NaN,NaN,14384469_2495,6265,4_0484_01_00168,,,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...
214,Places,Шацкий район,Шацк,район,,Shatsk,NaN,NaN,13569102_15,1512,3_3229_50_00007,,,"51.497793686969075, 23.93153500198429",https://uk.wikipedia.org/wiki/%D0%A8%D0%B0%D1%...
182,Places,Львовско-Сельский район,Львов,сельский район,,Lviv,"<>,Lwow,Lwow,Poland",5704942.0,13569102_4,1501,3_3229_35_00063,,,"49.842402697674146, 24.028025363350697",https://uk.wikipedia.org/wiki/%D0%91%D1%80%D1%...
86,description_02_uk,Кременецький повіт,Кременець,повіт,Krzemieniec,,NaN,NaN,14384469_2513,6283,4_0484_01_00186,7530489,"<>,Krzemieniec,Wolyn,Poland",NaN,NaN
178,description_02_uk,Лановецький район,Ланівці,район,,Lanivtsi,NaN,NaN,14384469_2525,6295,4_0484_01_00198,,,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...
175,description_02_uk,Лановецький район,Ланівці,район,,Lanivtsi,NaN,NaN,14384469_2522,6292,4_0484_01_00195,,,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...


### sample of 20 missing rows

,comb_id,Act_ID,Sign,source,ORIG_DISTRICT_STRING_CYR,NORM_DISTRICT_NAME_CYR,NORM_DISTRICT_TYPE_CYR,NORM_DISTRICT_NAME_ENG,NORM_DISTRICT_TYPE_ENG,MISS_CENTER_GPS,MISS_REF1,parachronistic_district_sapir,parachronistic_id_sapir
210,13569102_15,1512,3_3229_50_00007,Places,Любашевский район,Любешов,район,Lyubeshiv,rayon,"51.760681859517355, 25.4985159471312",https://uk.wikipedia.org/wiki/%D0%9B%D1%8E%D0%...,NaN,NaN
148,14384469_2495,6265,4_0484_01_00168,description_02_uk,Лановецький район,Ланівці,район,Lanivtsi,rayon,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...,NaN,NaN
186,13569102_12,1510,3_3229_49_00010,Places,Мизочский район,Мизоч,район,Myzoch,rayon,"50.40235303599611, 26.140663952191854",https://uk.wikipedia.org/wiki/%D0%9C%D1%96%D0%...,NaN,NaN
21,13569102_15,1512,3_3229_50_00007,Provenance,Ковельский район,Ковель,район,Kovel,rayon,"51.21356644353112, 24.70214310116599",https://uk.wikipedia.org/wiki/%D0%9A%D0%BE%D0%...,"<>,Kowel,Wolyn,Poland",5700239.0
124,14384469_2471,6241,4_0484_01_00144,description_02_uk,Лановецький район,Ланівці,район,Lanivtsi,rayon,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...,NaN,NaN
142,14384469_2489,6259,4_0484_01_00162,description_02_uk,Лановецький район,Ланівці,район,Lanivtsi,rayon,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...,NaN,NaN
13,13569102_14,1511,3_3229_50_00006,Places,Заболотьевский район,Заболотье,район,Zabolottia,rayon,"51.63359970448513, 24.241518274843195",https://uk.wikisource.org/wiki/%D0%97%D0%B0%D0...,NaN,NaN
191,13569102_7,1504,3_3229_48_00013,Provenance,Рівненський район,Рівне,район,Rivne,rayon,"50.61891491785878, 26.251030872482826",https://uk.wikipedia.org/wiki/%D0%A0%D1%96%D0%...,"<>,Rowne,Wolyn,Poland",5703885.0
182,13569102_4,1501,3_3229_35_00063,Places,Львовско-Сельский район,Львов,сельский район,Lviv,rural rayon,"49.842402697674146, 24.028025363350697",https://uk.wikipedia.org/wiki/%D0%91%D1%80%D1%...,"<>,Lwow,Lwow,Poland",5704942.0
183,13569102_15,1512,3_3229_50_00007,Places,Маневичский район,Маневичи,район,Manevychi,rayon,"51.29057652103595, 25.546945800868293",https://uk.wikipedia.org/wiki/%D0%9C%D0%B0%D0%...,NaN,NaN


In [ ]:
#@title REGION MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# loading transliteration dictionaries from json
with open("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/translit_dicts.json", "r", encoding="utf-8") as f:
    dicts = json.load(f)
base_dict = dicts["base_dict"]
extended_dict = dicts["extended_dict"]

# defining similarity threshold for fuzzy matching
similarity_threshold = 64

# defining function to build regex pattern from transliteration dictionary
def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alts = ext_dict[char]
            pattern += "(" + "|".join(re.escape(a) for a in alts) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# setting file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/final_compl_places_with_mach_transl_&_sapir_category.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# filtering sapir table for regions only
df_sapir = df_sapir[(df_sapir["place"] == "<>") & (df_sapir["district"] == "<>")].reset_index(drop=True)

# normalizing sapir region column
df_sapir["region_norm"] = df_sapir["region"].astype(str).str.lower().str.strip()

# normalizing final file columns
df_final["final_region_eng_norm"] = df_final["norm_admun_name_eng"].astype(str).str.lower().str.strip()
df_final_region = df_final[df_final["sapir_category"].astype(str).str.lower() == "region"].copy()
df_final_region["final_region_regex"] = df_final_region["norm_admun_name_cyr"].str.lower().map(
    lambda x: build_regex_pattern(str(x), extended_dict)
)

# initializing output and counters
output_rows = []
match_counts = {}
unique_regex_matches = set()
unique_fuzzy_matches = set()

# iterating over final regions
for _, row in df_final_region.iterrows():
    query_regex = row["final_region_regex"]
    query_eng = row["final_region_eng_norm"]
    source_first = row["norm_admun_name_cyr"][0].lower() if row["norm_admun_name_cyr"] else ""

    # performing base filtering
    filtered_base = [
        idx for idx, sr in df_sapir.iterrows()
        if source_first in base_dict and sr["region_norm"] and sr["region_norm"][0] in base_dict[source_first]
    ]

    # performing regex matching
    filtered_regex = [
        idx for idx in filtered_base
        if re.fullmatch(query_regex, df_sapir.at[idx, "region_norm"])
    ]

    if filtered_regex:
        final_cands = filtered_regex
        method = "regex"
    else:
        # performing fuzzy matching
        choices = df_sapir["region_norm"].tolist()
        matches = process.extractBests(
            query_eng, choices,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=similarity_threshold,
            limit=len(choices)
        )
        final_cands = []
        for m in matches:
            if len(m) == 3:
                final_cands.append(m[2])
            else:
                final_cands.extend([i for i, v in enumerate(choices) if v == m[0]])
        method = "fuzzy"

    # counting matches
    match_counts[row["final_region_eng_norm"]] = len(final_cands)
    val = row["final_region_eng_norm"]
    if final_cands:
        if method == "regex":
            unique_regex_matches.add(val)
        else:
            if val not in unique_regex_matches:
                unique_fuzzy_matches.add(val)

    # handling no-match cases
    if not final_cands:
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "ORIG_REGION_STRING_CYR": row["final_places"],
            "NORM_REGION_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_REGION_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_REGION_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_REGION_TYPE_ENG": row["norm_admun_type_eng"],
            "SAPIR_REGION_MATCH": "",
            "SAPIR_REGION_ID": "",
            "SAPIR_REGION_VALUE": "",
            "COUNT_MATCHES_BEF_MAN": 0,
            "CORRECTED": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "SAPIR_HISTORICALLY_INCOMPATIBLE": "",
            "MISS_CENTER_GPS": "",
            "MISS_REF1": "",
            "MISS_REF2": "",
            "parachronistic_id_sapir": "",
            "parachronistic_region_sapir": ""
        })

    # recording matched rows
    for idx in final_cands:
        sr = df_sapir.iloc[idx]
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "ORIG_REGION_STRING_CYR": row["final_places"],
            "NORM_REGION_NAME_CYR": row["norm_admun_name_cyr"],
            "NORM_REGION_TYPE_CYR": row["norm_admun_type_cyr"],
            "NORM_REGION_NAME_ENG": row["norm_admun_name_eng"],
            "NORM_REGION_TYPE_ENG": row["norm_admun_type_eng"],
            "SAPIR_REGION_MATCH": sr["region"],
            "SAPIR_REGION_ID": sr["book_id"],
            "SAPIR_REGION_VALUE": sr["TIENG"],
            "COUNT_MATCHES_BEF_MAN": len(final_cands),
            "CORRECTED": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "SAPIR_HISTORICALLY_INCOMPATIBLE": "",
            "MISS_CENTER_GPS": "",
            "MISS_REF1": "",
            "MISS_REF2": "",
            "parachronistic_id_sapir": "",
            "parachronistic_region_sapir": ""
        })

# building and saving output dataframe
df_output = pd.DataFrame(output_rows)
output_file = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_sapir_regex_&_fuzzy_match_region.xlsx"
df_output.to_excel(output_file, index=False)

# summarizing match results
total = df_final_region["final_region_eng_norm"].nunique()
with_matches = sum(1 for c in match_counts.values() if c > 0)
no_matches = total - with_matches
dist = {}
for c in match_counts.values():
    dist[c] = dist.get(c, 0) + 1
report = [
    f"total unique final_region_eng_norm values processed: {total}",
    f"values with one or more matches: {with_matches} ({with_matches/total*100:.2f}%)",
    f"values with no matches: {no_matches} ({no_matches/total*100:.2f}%)"
]
for c in sorted(dist):
    report.append(f"values with exactly {c} match(es): {dist[c]} ({dist[c]/total*100:.2f}%)")

# displaying summary report
display(Markdown("**summary report:**"))
display(Markdown("\n\n".join(report)))

# computing unique match statistics
unique_total = len(unique_regex_matches | unique_fuzzy_matches)
r_pct = (len(unique_regex_matches) / unique_total * 100) if unique_total else 0
f_pct = (len(unique_fuzzy_matches) / unique_total * 100) if unique_total else 0
display(Markdown("**unique match method statistics:**"))
display(Markdown(f"unique regex matches: {len(unique_regex_matches)} ({r_pct:.2f}%)"))
display(Markdown(f"unique fuzzy matches: {len(unique_fuzzy_matches)} ({f_pct:.2f}%)"))

# displaying random matched samples
df_matches = df_output[df_output["COUNT_MATCHES_BEF_MAN"] > 0].drop_duplicates(
    subset=["ORIG_REGION_STRING_CYR", "NORM_REGION_NAME_ENG"]
)
sample_df = df_matches.sample(n=20, random_state=42) if len(df_matches) >= 20 else df_matches
display(Markdown("**20 random sample rows with matches:**"))
display(sample_df[[
    "ORIG_REGION_STRING_CYR", "NORM_REGION_NAME_CYR",
    "NORM_REGION_NAME_ENG", "SAPIR_REGION_MATCH",
    "SAPIR_REGION_ID", "SAPIR_REGION_VALUE",
    "COUNT_MATCHES_BEF_MAN"
]])
display(Markdown(f"**output file saved at:** {output_file}"))
display(Markdown("**region matching process completed successfully.**"))


**loading excel files...**

**summary report:**

total unique final_region_eng_norm values processed: 15

values with one or more matches: 8 (53.33%)

values with no matches: 7 (46.67%)

values with exactly 0 match(es): 7 (46.67%)

values with exactly 1 match(es): 7 (46.67%)

values with exactly 2 match(es): 1 (6.67%)

**unique match method statistics:**

unique regex matches: 0 (0.00%)

unique fuzzy matches: 8 (100.00%)

**20 random sample rows with matches:**

,ORIG_REGION_STRING_CYR,NORM_REGION_NAME_CYR,NORM_REGION_NAME_ENG,SAPIR_REGION_MATCH,SAPIR_REGION_ID,SAPIR_REGION_VALUE,COUNT_MATCHES_BEF_MAN
1,Волынская область,Волынь,Volyn,Wolyn,5703791,"<>,<>,Wolyn,Poland",1
3,Житомирська область,Житомир,Zhytomyr,Zhitomir,5703815,"<>,<>,Zhitomir,Ukraine (USSR)",1
4,Запорізька область,Запорізька,Zaporizhzhia,Zaporozhye,5703950,"<>,<>,Zaporozhye,Ukraine (USSR)",1
5,Луганська область,Луганськ,Luhansk,Murmansk,5703806,"<>,<>,Murmansk,Russia (USSR)",2
11,Тернопільська область,Тернопіль,Ternopil,Tarnopol,5703951,"<>,<>,Tarnopol,Poland",1
12,Харківська область,Харків,Kharkiv,Kharkov,5703882,"<>,<>,Kharkov,Ukraine (USSR)",1
16,Черновицкая область,Черновцы,Chernivtsi,Chernigov,5703886,"<>,<>,Chernigov,Ukraine (USSR)",1
17,Чернівецька область,Черновцы,Chernivtsi,Chernigov,5703886,"<>,<>,Chernigov,Ukraine (USSR)",1
18,Чернігівськіа область,Чернігів,Chernihiv,Chernigov,5703886,"<>,<>,Chernigov,Ukraine (USSR)",1


**output file saved at:** /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_sapir_regex_&_fuzzy_match_region.xlsx

**region matching process completed successfully.**

In [ ]:
#@title UNPIVOTED REGIONS WITH SAPIR

# defining directories and filenames
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"
mapping_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"

manual_file = "compl_places_sapir_regex_&_fuzzy_match_region_with_man.xlsx"
mapping_file = "t_General_Act_IDs_with_old_ID.xlsx"

# loading dataframes
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(mapping_dir, mapping_file))

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# converting mapping id fields to string uniformly
df_mapping["Comb_ID"] = df_mapping["Comb_ID"].apply(normalize_id).astype("string")
df_mapping["Act_ID"]  = df_mapping["Act_ID"].apply(normalize_id).astype("string")
df_mapping["Sign"]    = df_mapping["Sign"].apply(normalize_id).astype("string")

# filtering out removed rows
df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

# preparing new columns for output
df_manual["source"]                        = df_manual["source"]
df_manual["region_cyr"]                    = df_manual["ORIG_REGION_STRING_CYR"]
df_manual["norm_region_name_cyr"]          = df_manual["NORM_REGION_NAME_CYR"]
df_manual["norm_region_type_cyr"]          = df_manual["NORM_REGION_TYPE_CYR"]
df_manual["region_sapir"]                  = df_manual["SAPIR_REGION_MATCH"].where(df_manual["STATUS"] == "to use", "")
df_manual["region_missing"]                = df_manual["NORM_REGION_NAME_ENG"].where(df_manual["STATUS"] == "to export", "")
df_manual["full_sapir"]                    = df_manual["SAPIR_REGION_VALUE"].where(df_manual["STATUS"] == "to use", "")
df_manual["parachronistic_region_sapir"]   = df_manual["parachronistic_region_sapir"]
df_manual["parachronistic_id_sapir"]       = (df_manual["parachronistic_id_sapir"].apply(normalize_id).astype("string"))

# splitting and exploding comb_id lists
df_manual["comb_id_list"] = df_manual["orig_incom&id"].astype(str).apply(
    lambda s: [normalize_id(item) for item in s.split(",") if item.strip()]
)
df_exploded = df_manual.explode("comb_id_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"comb_id_list": "comb_id"})
df_exploded["comb_id"] = df_exploded["comb_id"].astype("string")

# merging with mapping on comb_id
df_merged = pd.merge(
    df_exploded,
    df_mapping[["Comb_ID", "Act_ID", "Sign"]],
    left_on="comb_id",
    right_on="Comb_ID",
    how="left"
)

# filling missing Act_ID/Sign with placeholder
df_merged["Act_ID"] = df_merged["Act_ID"].fillna("missing").astype("string")
df_merged["Sign"]   = df_merged["Sign"].fillna("missing").astype("string")

# normalizing sapir_id and filtering by STATUS
df_merged["sapir_id"] = df_merged["SAPIR_REGION_ID"].apply(normalize_id).astype("string")
df_merged["sapir_id"] = df_merged["sapir_id"].where(df_merged["STATUS"] == "to use", "")
df_merged["full_sapir"] = df_merged["full_sapir"].where(df_merged["STATUS"] == "to use", "")

# preparing final unpivoted output
cols_output = [
    "source",
    "region_cyr",
    "norm_region_name_cyr",
    "norm_region_type_cyr",
    "region_sapir",
    "region_missing",
    "parachronistic_region_sapir",
    "parachronistic_id_sapir",
    "comb_id",
    "Act_ID",
    "Sign",
    "sapir_id",
    "full_sapir",
    "MISS_CENTER_GPS",
    "MISS_REF1",
    "MISS_REF2"
]
df_output = df_merged[cols_output].copy()

# preparing missing sapir regions output by STATUS == "to export"
df_missing = df_merged[df_merged["STATUS"] == "to export"][[
    "comb_id",
    "source",
    "Act_ID",
    "Sign",
    "ORIG_REGION_STRING_CYR",
    "NORM_REGION_NAME_CYR",
    "NORM_REGION_TYPE_CYR",
    "NORM_REGION_NAME_ENG",
    "NORM_REGION_TYPE_ENG",
    "MISS_CENTER_GPS",
    "MISS_REF1",
    "MISS_REF2",
    "parachronistic_id_sapir",
    "parachronistic_region_sapir"
]].drop_duplicates(subset=["comb_id", "ORIG_REGION_STRING_CYR"])

# computing statistics
total_manual = len(df_manual)
total_exploded = len(df_output)
perc_explosion = total_exploded / total_manual * 100 if total_manual else 0
source_counts = df_output["source"].value_counts()
source_perc = source_counts / source_counts.sum() * 100
unique_regions = df_output["region_cyr"].nunique()
unique_by_src = df_output.groupby("source")["region_cyr"].nunique()
status_counts = df_manual["STATUS"].value_counts()
missing_map_count = df_output[(df_output["Act_ID"] == "missing") & (df_output["Sign"] == "missing")].shape[0]
missing_map_pct   = missing_map_count / total_exploded * 100 if total_exploded else 0
total_missing      = len(df_missing)
gps_dashes        = df_missing["MISS_CENTER_GPS"].astype(str).str.contains("---").sum()
perc_gps_dashes   = gps_dashes / total_missing * 100 if total_missing else 0

# building summary
source_lines = "\n".join(f"- {src}: {cnt} ({source_perc[src]:.2f}%)"
                         for src, cnt in source_counts.items())
unique_lines = "\n".join(f"- {src}: {cnt}"
                         for src, cnt in unique_by_src.items())
status_line = ", ".join(f"{st}: {cnt}"
                         for st, cnt in status_counts.items())

summary_md = f"""
## summary report

- **total manual rows:** {total_manual}
- **total exploded region rows:** {total_exploded} ({perc_explosion:.2f}%)

### source breakdown
{source_lines}

- **unique region_cyr overall:** {unique_regions}

### unique region_cyr by source
{unique_lines}

- **manual status counts:** {status_line}

- **mapping missing comb_ids:** {missing_map_count} ({missing_map_pct:.2f}%)
- **missing sapir region rows:** {total_missing}, dashes in GPS: {gps_dashes} ({perc_gps_dashes:.2f}%)
"""

# displaying summary in markdown
display(Markdown(summary_md))

# saving outputs
df_output.to_excel(os.path.join(file_dir, "compl_places_unpivoted_with_sapir_region.xlsx"), index=False)
df_missing.to_excel(os.path.join(file_dir, "compl_places_missing_in_sapir_region.xlsx"), index=False)

# showing sample sections
display(Markdown("### sample of 20 unpivoted region rows"))
display(df_output.sample(n=20, random_state=42) if total_exploded >= 20 else df_output)
display(Markdown("### sample of 20 missing region rows"))
display(df_missing.sample(n=20, random_state=42) if total_missing >= 20 else df_missing)


## summary report

- **total manual rows:** 19
- **total exploded region rows:** 523 (2752.63%)

### source breakdown
- creator: 369 (70.55%)
- description_02_uk: 136 (26.00%)
- Places: 17 (3.25%)
- ru_details_mod: 1 (0.19%)

- **unique region_cyr overall:** 17

### unique region_cyr by source
- Places: 5
- creator: 11
- description_02_uk: 1
- ru_details_mod: 1

- **manual status counts:** to export: 11, to use: 8

- **mapping missing comb_ids:** 40 (7.65%)
- **missing sapir region rows:** 469, dashes in GPS: 0 (0.00%)


### sample of 20 unpivoted region rows

,source,region_cyr,norm_region_name_cyr,norm_region_type_cyr,region_sapir,region_missing,parachronistic_region_sapir,parachronistic_id_sapir,comb_id,Act_ID,Sign,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1,MISS_REF2
521,creator,Луганська область,Луганськ,область,,Luhansk,NaN,,13439437_36,1492,1_X_01_06721,,,"48.570556888262736, 39.32409228393431",https://uk.wikipedia.org/wiki/%D0%9B%D1%83%D0%...,NaN
388,description_02_uk,Тернопільська область,Тернопіль,область,,Ternopil,"<>,<>,Tarnopol,Poland",5703951,14384469_3093,6940,4_R-0205_01_00012,,,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,NaN
6,creator,Львівська область,Львов,область,,Lviv,"<>,<>,Lwow,Poland",5703942,13439437_82,5,1_0005_01_00005,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN
184,creator,Львівська область,Львов,область,,Lviv,"<>,<>,Lwow,Poland",5703942,13439437_260,164,1_0005_01_00165,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN
78,creator,Львівська область,Львов,область,,Lviv,"<>,<>,Lwow,Poland",5703942,13439437_154,67,1_0005_01_00068,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN
299,Places,Львовская область,Львов,область,,Lviv,"<>,<>,Lwow,Poland",5703942,13569102_4,1501,3_3229_35_00063,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN
522,creator,Луганська область,Луганськ,область,,Luhansk,NaN,,13439437_36,1492,1_X_01_06721,,,"48.570556888262736, 39.32409228393431",https://uk.wikipedia.org/wiki/%D0%9B%D1%83%D0%...,NaN
485,description_02_uk,Тернопільська область,Тернопіль,область,,Ternopil,"<>,<>,Tarnopol,Poland",5703951,14384469_3190,7039,4_R-0274_01_00096,,,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,NaN
117,creator,Львівська область,Львов,область,,Lviv,"<>,<>,Lwow,Poland",5703942,13439437_193,101,1_0005_01_00102,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN
137,creator,Львівська область,Львов,область,,Lviv,"<>,<>,Lwow,Poland",5703942,13439437_213,121,1_0005_01_00122,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN


### sample of 20 missing region rows

,comb_id,source,Act_ID,Sign,ORIG_REGION_STRING_CYR,NORM_REGION_NAME_CYR,NORM_REGION_TYPE_CYR,NORM_REGION_NAME_ENG,NORM_REGION_TYPE_ENG,MISS_CENTER_GPS,MISS_REF1,MISS_REF2,parachronistic_id_sapir,parachronistic_region_sapir
55,13439437_131,creator,48,1_0005_01_00049,Львівська область,Львов,область,Lviv,oblast,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN,5703942,"<>,<>,Lwow,Poland"
73,13439437_149,creator,missing,missing,Львівська область,Львов,область,Lviv,oblast,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN,5703942,"<>,<>,Lwow,Poland"
33,13439437_109,creator,30,1_0005_01_00031,Львівська область,Львов,область,Lviv,oblast,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN,5703942,"<>,<>,Lwow,Poland"
490,14384469_3195,description_02_uk,7044,4_R-0274_01_00101,Тернопільська область,Тернопіль,область,Ternopil,oblast,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,NaN,5703951,"<>,<>,Tarnopol,Poland"
470,14384469_3175,description_02_uk,7024,4_R-0274_01_00081,Тернопільська область,Тернопіль,область,Ternopil,oblast,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,NaN,5703951,"<>,<>,Tarnopol,Poland"
229,13439437_305,creator,202,1_0005_01_00203,Львівська область,Львов,область,Lviv,oblast,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN,5703942,"<>,<>,Lwow,Poland"
210,13439437_286,creator,184,1_0005_01_00185,Львівська область,Львов,область,Lviv,oblast,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN,5703942,"<>,<>,Lwow,Poland"
9,13439437_85,creator,8,1_0005_01_00008,Львівська область,Львов,область,Lviv,oblast,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN,5703942,"<>,<>,Lwow,Poland"
397,14384469_3102,description_02_uk,6951,4_R-0274_01_00008,Тернопільська область,Тернопіль,область,Ternopil,oblast,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,NaN,5703951,"<>,<>,Tarnopol,Poland"
70,13439437_146,creator,missing,missing,Львівська область,Львов,область,Lviv,oblast,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,NaN,5703942,"<>,<>,Lwow,Poland"


In [ ]:
#@title COUNTRY MATCHING WITH SAPIR TABLE - REGEX & FUZZY MATCH

# loading transliteration dictionaries from json
with open("/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/translit_dicts.json", "r", encoding="utf-8") as f:
    dicts = json.load(f)
base_dict = dicts["base_dict"]
extended_dict = dicts["extended_dict"]

# defining similarity threshold for fuzzy matching
similarity_threshold = 85

# defining function to build regex pattern from transliteration dictionary
def build_regex_pattern(query_cyr, ext_dict):
    pattern = ""
    for char in query_cyr:
        if char in ext_dict:
            alts = ext_dict[char]
            pattern += "(" + "|".join(re.escape(a) for a in alts) + ")"
        else:
            pattern += re.escape(char)
    return pattern

# setting file paths
final_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/final_compl_places_with_mach_transl_&_sapir_category.xlsx"
sapir_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/SAPIR_TABLES/t_SC_Places_Items(ENG_HEB)_Values.xlsx"

# loading excel files
display(Markdown("**loading excel files...**"))
df_final = pd.read_excel(final_path)
df_sapir = pd.read_excel(sapir_path)

# filtering sapir table for countries only
df_sapir = df_sapir[
    (df_sapir["place"] == "<>") &
    (df_sapir["district"] == "<>") &
    (df_sapir["region"] == "<>")
].reset_index(drop=True)

# normalizing sapir country column
df_sapir["country_norm"] = df_sapir["country"].astype(str).str.lower().str.strip()

# normalize final file columns
df_final["final_country_eng_norm"] = df_final["final_places_eng"].astype(str).str.lower().str.strip()
df_final_country = df_final[df_final["sapir_category"].astype(str).str.lower() == "country"].copy()

# build regex from final_places (cyrillic)
df_final_country["final_country_regex"] = df_final_country["final_places"].astype(str).str.lower().map(
    lambda x: build_regex_pattern(x, extended_dict)
)

# initializing output and counters
output_rows = []
match_counts = {}

# iterating over final countries
for _, row in df_final_country.iterrows():
    cyr = str(row["final_places"]).lower().strip()
    first = cyr[0] if cyr else ""
    regex = row["final_country_regex"]
    eng = row["final_country_eng_norm"]

    # base filtering by first letter transliteration
    base_idxs = [
        i for i, r in df_sapir.iterrows()
        if first in base_dict and r["country_norm"] and r["country_norm"][0] in base_dict[first]
    ]

    # regex match
    reg_idxs = [i for i in base_idxs if re.fullmatch(regex, df_sapir.at[i, "country_norm"])]

    if reg_idxs:
        cands = reg_idxs
    else:
        # fuzzy match
        choices = df_sapir["country_norm"].tolist()
        matches = process.extractBests(
            eng,
            choices,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=similarity_threshold,
            limit=len(choices)
        )
        cands = []
        for m in matches:
            if len(m) == 3:
                cands.append(m[2])
            else:
                cands.extend([i for i, v in enumerate(choices) if v == m[0]])

    match_counts[eng] = len(cands)

    # if no candidates, record empty
    if not cands:
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "ORIG_COUNTRY_STRING_CYR": row["final_places"],
            "SAPIR_COUNTRY_MATCH": "",
            "SAPIR_COUNTRY_ID": "",
            "SAPIR_COUNTRY_VALUE": "",
            "COUNT_MATCHES_BEF_MAN": 0,
            "CORRECTED": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISS_CENTER_GPS": "",
            "MISS_REF1": "",
            "MISS_REF2": ""
        })

    # record each match
    for i in cands:
        sap = df_sapir.iloc[i]
        output_rows.append({
            "orig_incom&id": row["orig_incom&id"],
            "source": row["source"],
            "ORIG_COUNTRY_STRING_CYR": row["final_places"],
            "SAPIR_COUNTRY_MATCH": sap["country"],
            "SAPIR_COUNTRY_ID": sap["book_id"],
            "SAPIR_COUNTRY_VALUE": sap["TIENG"],
            "COUNT_MATCHES_BEF_MAN": len(cands),
            "CORRECTED": "",
            "STATUS": "",
            "MISSING_IN_SAPIR": "",
            "MISS_CENTER_GPS": "",
            "MISS_REF1": "",
            "MISS_REF2": ""
        })

# build and save output
df_output = pd.DataFrame(output_rows)
out_path = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_sapir_regex_&_fuzzy_match_country.xlsx"
df_output.to_excel(out_path, index=False)

# summary report
from collections import Counter
total = df_final_country["final_country_eng_norm"].nunique()
with_matches = sum(1 for v in match_counts.values() if v > 0)
no_matches = total - with_matches
dist = Counter(match_counts.values())

summary = [
    f"total unique final_country_eng_norm: {total}",
    f"with matches: {with_matches} ({with_matches/total*100:.2f}%)",
    f"no matches: {no_matches} ({no_matches/total*100:.2f}%)"
]
for c, cnt in sorted(dist.items()):
    summary.append(f"{cnt} values with {c} match(es) ({cnt/total*100:.2f}%)")

display(Markdown("**summary report:**"))
display(Markdown("\n\n".join(summary)))

# sample output
display(Markdown("**sample matched rows:**"))
display(df_output.sample(n=10) if len(df_output) >= 10 else df_output)

display(Markdown(f"**file saved to:** {out_path}"))


**loading excel files...**

**summary report:**

total unique final_country_eng_norm: 3

with matches: 0 (0.00%)

no matches: 3 (100.00%)

3 values with 0 match(es) (100.00%)

**sample matched rows:**

,orig_incom&id,source,ORIG_COUNTRY_STRING_CYR,SAPIR_COUNTRY_MATCH,SAPIR_COUNTRY_ID,SAPIR_COUNTRY_VALUE,COUNT_MATCHES_BEF_MAN,CORRECTED,STATUS,MISSING_IN_SAPIR,MISS_CENTER_GPS,MISS_REF1,MISS_REF2
0,"13569102_1, 13569102_2, 13569102_3, 13569102_4...",Provenance,УРСР,,,,0,,,,,,
1,11803151_22,ru_details_mod,Украина,,,,0,,,,,,
2,"13569102_1, 13569102_2, 13569102_3, 13569102_4...",Provenance,Українa,,,,0,,,,,,
3,"13439437_37, 13439437_38",creator,Українa,,,,0,,,,,,


**file saved to:** /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_sapir_regex_&_fuzzy_match_country.xlsx

In [ ]:
#@title UNPIVOTED COUNTRIES WITH SAPIR

# defining directories and filenames

file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"
mapping_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"

manual_file = "compl_places_sapir_regex_&_fuzzy_match_country_with_man.xlsx"
mapping_file = "t_General_Act_IDs_with_old_ID.xlsx"

# loading dataframes

df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(mapping_dir, mapping_file))

# defining a function to normalize id values

def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# converting mapping id fields to uniform string

df_mapping["Comb_ID"] = df_mapping["Comb_ID"].apply(normalize_id).astype("string")
df_mapping["Act_ID"] = df_mapping["Act_ID"].apply(normalize_id).astype("string")
df_mapping["Sign"] = df_mapping["Sign"].apply(normalize_id).astype("string")

# filtering out rows marked for removal

df_manual = df_manual[df_manual["STATUS"] != "to remove"].copy()

# preparing new columns for output

df_manual["source"] = df_manual["source"]
df_manual["country_cyr"] = df_manual["ORIG_COUNTRY_STRING_CYR"]
df_manual["country_sapir"] = df_manual["SAPIR_COUNTRY_MATCH"].where(df_manual["STATUS"] == "to use", "")
df_manual["full_sapir"] = df_manual["SAPIR_COUNTRY_VALUE"].where(df_manual["STATUS"] == "to use", "")

# splitting and exploding comb_id lists

df_manual["comb_id_list"] = df_manual["orig_incom&id"].astype(str).apply(
    lambda s: [normalize_id(item) for item in s.split(",") if item.strip()]
)
df_exploded = df_manual.explode("comb_id_list").reset_index(drop=True)
df_exploded = df_exploded.rename(columns={"comb_id_list": "comb_id"})
df_exploded["comb_id"] = df_exploded["comb_id"].astype("string")

# merging with mapping by comb_id

df_merged = pd.merge(
    df_exploded,
    df_mapping[["Comb_ID", "Act_ID", "Sign"]],
    left_on="comb_id",
    right_on="Comb_ID",
    how="left"
)

# filling missing Act_ID and Sign with placeholder

df_merged["Act_ID"] = df_merged["Act_ID"].fillna("missing").astype("string")
df_merged["Sign"] = df_merged["Sign"].fillna("missing").astype("string")

# normalizing sapir_id from the sapir country id column

df_merged["sapir_id"] = df_merged["SAPIR_COUNTRY_ID"].apply(normalize_id).astype("string")

# selecting final columns for output

cols_output = [
    "source",
    "country_cyr",
    "country_sapir",
    "comb_id",
    "Act_ID",
    "Sign",
    "sapir_id",
    "full_sapir"
]
df_output = df_merged[cols_output].copy()

# computing basic statistics

total_manual = len(df_manual)
total_exploded = len(df_output)
perc_explosion = total_exploded / total_manual * 100 if total_manual else 0
source_counts = df_output["source"].value_counts()
unique_countries = df_output["country_cyr"].nunique()

# building summary report

summary_md = f"""
## summary report

- **total manual rows:** {total_manual}
- **total exploded country rows:** {total_exploded} ({perc_explosion:.2f}%)

### source breakdown
""" + "\n".join(f"- {src}: {cnt}" for src, cnt in source_counts.items()) + f"""

- **unique country_cyr overall:** {unique_countries}
"""

display(Markdown(summary_md))

# saving the unpivoted table

df_output.to_excel(
    os.path.join(file_dir, "compl_places_unpivoted_with_sapir_country.xlsx"),
    index=False
)

# showing a sample of unpivoted rows

display(Markdown("### sample of 20 unpivoted country rows"))
display(df_output.sample(n=20, random_state=42) if total_exploded >= 20 else df_output)



## summary report

- **total manual rows:** 4
- **total exploded country rows:** 37 (925.00%)

### source breakdown
- Provenance: 34
- creator: 2
- ru_details_mod: 1

- **unique country_cyr overall:** 3


### sample of 20 unpivoted country rows

,source,country_cyr,country_sapir,comb_id,Act_ID,Sign,sapir_id,full_sapir
17,ru_details_mod,Украина,Ukraine (USSR),11803151_22,8349,6_R-3009_14_01658,5459048,"<>,<>,<>,Ukraine (USSR)"
13,Provenance,УРСР,Ukraine (USSR),13569102_14,1511,3_3229_50_00006,5459048,"<>,<>,<>,Ukraine (USSR)"
4,Provenance,УРСР,Ukraine (USSR),13569102_5,1502,3_3229_37_00018,5459048,"<>,<>,<>,Ukraine (USSR)"
29,Provenance,Українa,Ukraine (USSR),13569102_12,1510,3_3229_49_00010,5459048,"<>,<>,<>,Ukraine (USSR)"
35,creator,Українa,Ukraine (USSR),13439437_37,1494,1_X_01_08864,5459048,"<>,<>,<>,Ukraine (USSR)"
25,Provenance,Українa,Ukraine (USSR),13569102_8,1505,3_3229_48_00014,5459048,"<>,<>,<>,Ukraine (USSR)"
6,Provenance,УРСР,Ukraine (USSR),13569102_7,1504,3_3229_48_00013,5459048,"<>,<>,<>,Ukraine (USSR)"
26,Provenance,Українa,Ukraine (USSR),13569102_9,1506,3_3229_48_00015,5459048,"<>,<>,<>,Ukraine (USSR)"
24,Provenance,Українa,Ukraine (USSR),13569102_7,1504,3_3229_48_00013,5459048,"<>,<>,<>,Ukraine (USSR)"
16,Provenance,УРСР,Ukraine (USSR),13569102_17,1514,3_3229_53_00008,5459048,"<>,<>,<>,Ukraine (USSR)"


In [ ]:
#@title UNPIVOTED ADMINISTRATIVE_DIVISION

# defining directories and filenames
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"
mapping_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"

manual_file = "compl_places_missing_in_sapir_administrative_division_bef_processing.xlsx"
mapping_file = "t_General_Act_IDs_with_old_ID.xlsx"

# loading dataframes
df_manual = pd.read_excel(os.path.join(file_dir, manual_file))
df_mapping = pd.read_excel(os.path.join(mapping_dir, mapping_file))

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# normalizing mapping id fields
for col in ["Comb_ID", "Act_ID", "Sign"]:
    df_mapping[col] = df_mapping[col].apply(normalize_id).astype("string")

# preparing new columns in df_manual
df_manual["source"] = df_manual["source"]
df_manual["admdiv_cyr"] = df_manual["ADMDIV_STRING_CYR"]
df_manual["norm_admdiv_name_cyr"] = df_manual["NORM_ADMDIV_NAME_CYR"]
df_manual["norm_admdiv_type_cyr"] = df_manual["NORM_ADMDIV_TYPE_CYR"]
df_manual["admdiv_sapir"] = ""
df_manual["admdiv_missing"] = df_manual["ADMDIV_STRING_ENG"]
df_manual["norm_admdiv_name_eng"] = df_manual["NORM_ADMDIV_NAME_ENG"]
df_manual["norm_admdiv_type_eng"] = df_manual["NORM_ADMDIV_TYPE_ENG"]
df_manual["MISS_CENTER_GPS"] = df_manual["MISS_CENTER_GPS"]
df_manual["MISS_REF1"] = df_manual["MISS_REF1"]
df_manual["MISS_REF2"] = df_manual["MISS_REF2"]

# splitting and exploding comb_id
df_manual["comb_id_list"] = df_manual["comb_id"].astype(str).apply(
    lambda s: [normalize_id(i) for i in s.split(",") if i.strip()]
)
df_manual = df_manual.drop(columns=["comb_id"])
df_exploded = df_manual.explode("comb_id_list").rename(columns={"comb_id_list": "comb_id"})
df_exploded["comb_id"] = df_exploded["comb_id"].astype("string")

# merging with mapping on comb_id
df_merged = pd.merge(
    df_exploded,
    df_mapping[["Comb_ID", "Act_ID", "Sign"]],
    left_on="comb_id",
    right_on="Comb_ID",
    how="left"
)

# filling missing Act_ID and Sign
df_merged["Act_ID"] = df_merged["Act_ID"].fillna("missing")
df_merged["Sign"] = df_merged["Sign"].fillna("missing")

# preparing final unpivoted output
df_merged["sapir_id"] = ""
df_merged["full_sapir"] = ""
cols_output = [
    "source",
    "admdiv_cyr",
    "norm_admdiv_name_cyr",
    "norm_admdiv_type_cyr",
    "admdiv_sapir",
    "admdiv_missing",
    "comb_id",
    "Act_ID",
    "Sign",
    "sapir_id",
    "full_sapir",
    "MISS_CENTER_GPS",
    "MISS_REF1"
]
df_output = df_merged[cols_output].copy()

# computing summary statistics
total_manual = len(df_manual)
total_exploded = len(df_output)
perc_explosion = total_exploded / total_manual * 100 if total_manual else 0
source_counts = df_output["source"].value_counts()
source_perc = source_counts / source_counts.sum() * 100
unique_admdiv = df_output["admdiv_cyr"].nunique()
missing_act = (df_output["Act_ID"] == "missing").sum()
missing_sign = (df_output["Sign"] == "missing").sum()

# outputting summary
source_lines = "\n".join(f"- {src}: {cnt} ({source_perc[src]:.2f}%)" for src, cnt in source_counts.items())

summary_md = f"""
**summary report**

- total manual rows: {total_manual}
- total exploded rows: {total_exploded} ({perc_explosion:.2f}%)

**source breakdown**
{source_lines}

**unique admdiv_cyr overall**
- {unique_admdiv}

**missing ID counts**
- missing act_id: {missing_act}
- missing sign: {missing_sign}
"""
display(Markdown(summary_md))

# saving unpivoted output file
output_file = os.path.join(file_dir, "compl_places_unpivoted_with_sapir_admdiv.xlsx")
df_output.to_excel(output_file, index=False)
print(f"\noutput file saved at: {output_file}")

# preparing and saving missing sapir admin-division output
df_missing = df_merged[[
    "comb_id",
    "source",
    "Act_ID",
    "Sign",
    "ADMDIV_STRING_CYR",
    "NORM_ADMDIV_NAME_CYR",
    "NORM_ADMDIV_TYPE_CYR",
    "ADMDIV_STRING_ENG",
    "NORM_ADMDIV_NAME_ENG",
    "NORM_ADMDIV_TYPE_ENG",
    "MISS_CENTER_GPS",
    "MISS_REF1",
    "MISS_REF2"
]].drop_duplicates(subset=["comb_id", "ADMDIV_STRING_CYR"])
missing_file = os.path.join(file_dir, "compl_places_missing_in_sapir_administrative_division.xlsx")
df_missing.to_excel(
    os.path.join(file_dir, "compl_places_missing_in_sapir_administrative_division.xlsx"),
    index=False
)

# displaying sample rows
display(df_output.sample(n=20, random_state=42) if total_exploded >= 20 else df_output)
print(f"\n\nmissing file saved at: {missing_file}")
display(df_missing.sample(n=20, random_state=42) if total_exploded >= 20 else df_output)



**summary report**

- total manual rows: 1
- total exploded rows: 4 (400.00%)

**source breakdown**
- Provenance: 4 (100.00%)

**unique admdiv_cyr overall**
- 1

**missing ID counts**
- missing act_id: 0
- missing sign: 0



output file saved at: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_unpivoted_with_sapir_admdiv.xlsx


,source,admdiv_cyr,norm_admdiv_name_cyr,norm_admdiv_type_cyr,admdiv_sapir,admdiv_missing,comb_id,Act_ID,Sign,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1
0,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_1,1498,3_3229_35_00058,,,"49.84107424234717, 24.01567181048781",NaN
1,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_2,1499,3_3229_35_00059,,,"49.84107424234717, 24.01567181048781",NaN
2,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_3,1500,3_3229_35_00062,,,"49.84107424234717, 24.01567181048781",NaN
3,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_4,1501,3_3229_35_00063,,,"49.84107424234717, 24.01567181048781",NaN




missing file saved at: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_missing_in_sapir_administrative_division.xlsx


,source,admdiv_cyr,norm_admdiv_name_cyr,norm_admdiv_type_cyr,admdiv_sapir,admdiv_missing,comb_id,Act_ID,Sign,sapir_id,full_sapir,MISS_CENTER_GPS,MISS_REF1
0,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_1,1498,3_3229_35_00058,,,"49.84107424234717, 24.01567181048781",NaN
1,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_2,1499,3_3229_35_00059,,,"49.84107424234717, 24.01567181048781",NaN
2,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_3,1500,3_3229_35_00062,,,"49.84107424234717, 24.01567181048781",NaN
3,Provenance,Львівський евакуаційний район,Львів,евакуаційний район,,Lviv evacuation rayon,13569102_4,1501,3_3229_35_00063,,,"49.84107424234717, 24.01567181048781",NaN


In [ ]:
#@title MERGING UNPIVOTED

# loading base directory and file paths
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"
coordination_file = os.path.join(base_dir, "compl_unpivoted_union_table.xlsx")
output_file = os.path.join(base_dir, "compl_places_unpivoted_with_sapir_unified.xlsx")

# loading mapping configuration
df_mapping = pd.read_excel(coordination_file)

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# defining final columns for unified dataframe
final_columns = [c for c in df_mapping.columns if c != "FILE_NAME"]

# preparing logging structures
df_list = []
file_row_counts = {}
missing_files = []
missing_columns = {fn: [] for fn in df_mapping["FILE_NAME"]}

# iterating over mapping rows
for idx, row in df_mapping.iterrows():
    file_name = row["FILE_NAME"]
    source_path = os.path.join(base_dir, file_name)
    # loading source file
    try:
        df_source = pd.read_excel(source_path)
    except Exception:
        missing_files.append(file_name)
        print(f"file not found: {file_name}")
        continue
    # creating unified dataframe structure
    df_new = pd.DataFrame(index=df_source.index, columns=final_columns)
    # filling df_new based on mapping
    for col in final_columns:
        if col == "CATEGORY_SAPIR":
            df_new[col] = row[col] if pd.notnull(row[col]) else ""
        else:
            mapping_val = row[col] if pd.notnull(row[col]) else ""
            if not mapping_val:
                df_new[col] = ""
            elif mapping_val in df_source.columns:
                df_new[col] = df_source[mapping_val]
            else:
                df_new[col] = ""
                missing_columns[file_name].append(mapping_val)
                print(f"{file_name}: column not found {mapping_val}")
    # recording row count and appending dataframe
    file_row_counts[file_name] = len(df_new)
    df_list.append(df_new)
    print(f"processed {file_name}, rows: {len(df_new)}")

# concatenating dataframes
if df_list:
    df_union = pd.concat(df_list, ignore_index=True)
else:
    df_union = pd.DataFrame(columns=final_columns)

# logging missing files and columns
print(f"missing files: {missing_files}")
for fn, cols in missing_columns.items():
    if cols:
        print(f"{fn} — missing columns: {cols}")

# normalize ID_PARACHR_SAPIR
if "ID_PARACHR_SAPIR" in df_union.columns:
    df_union["ID_PARACHR_SAPIR"] = df_union["ID_PARACHR_SAPIR"].apply(normalize_id)

# converting id columns to string without decimals
id_cols = ["Comb_id", "Act_ID", "Sign", "ID_SAPIR"]
for col in id_cols:
    if col in df_union.columns:
        df_union[col] = df_union[col].apply(
            lambda x: str(int(float(x))) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit()
            else ("" if pd.isna(x) else x)
        )

# creating effective_name column for internal stats
df_union["effective_name"] = df_union["NAME_SAPIR"].where(
    df_union["NAME_SAPIR"].fillna("") != "", df_union["NAME_MISSING_SAPIR"]
)

# collecting summary statistics
total_rows = len(df_union)
rows_per_file = file_row_counts
source_counts = df_union["SOURCE"].value_counts()

# computing stats per category
category_stats = []
for cat in df_union["CATEGORY_SAPIR"].unique():
    df_cat = df_union[df_union["CATEGORY_SAPIR"] == cat]
    total_unique = df_cat["effective_name"].nunique()
    source_unique = df_cat.groupby("SOURCE")["effective_name"].nunique().to_dict()
    category_stats.append((cat, total_unique, source_unique))

# computing distribution of unique ID_SAPIR per Comb_id
dist_comb = df_union[df_union["ID_SAPIR"] != ""].groupby("Comb_id")["ID_SAPIR"].nunique()
dist_comb_counts = dist_comb.value_counts().sort_index()

# computing distribution of unique ID_SAPIR per Act_ID
dist_act = df_union[(df_union["Act_ID"] != "") & (df_union["ID_SAPIR"] != "")].groupby("Act_ID")["ID_SAPIR"].nunique()
dist_act_counts = dist_act.value_counts().sort_index()

# computing ID_SAPIR filled vs empty
sapir_filled = (df_union["ID_SAPIR"].fillna("") != "").sum()
sapir_empty = (df_union["ID_SAPIR"].fillna("") == "").sum()

# computing top 7 combinations of NAME_CYR and ID_SAPIR
combo_series = (
    df_union[(df_union["NAME_CYR"] != "") & (df_union["ID_SAPIR"] != "")]
    .assign(name_combo=lambda x: x["NAME_CYR"].astype(str) + " " + x["ID_SAPIR"].astype(str))
    ["name_combo"]
    .value_counts()
    .head(7)
)

# computing top 5 NAME_MISSING_SAPIR and top 5 NAME_PARACHR_SAPIR
top5_missing = df_union[df_union["NAME_MISSING_SAPIR"] != ""]["NAME_MISSING_SAPIR"].value_counts().head(5)
top5_parachr = df_union[df_union["NAME_PARACHR_SAPIR"] != ""]["NAME_PARACHR_SAPIR"].value_counts().head(5)

# computing GPS coordinates distribution for missing sapir entries
gps_df = df_union[["MISS_CENTER_GPS", "NAME_MISSING_SAPIR"]].drop_duplicates()
gps_provided = gps_df[gps_df["MISS_CENTER_GPS"] != "---"].shape[0]
gps_missing = gps_df[gps_df["MISS_CENTER_GPS"] == "---"].shape[0]

# preparing markdown report
lines = [
    "## summary report",
    f"- **total rows:** {total_rows}",
    "",
    "### rows per file",
    *[f"- {fn}: {cnt} ({cnt/total_rows*100:.2f}%)" for fn, cnt in rows_per_file.items()],
    "",
    "### source column stats",
    *[f"- **{src}:** {count} ({count/total_rows*100:.2f}%)" for src, count in source_counts.items()],
    "",
    "### stats per category (unique effective names)",
]
for cat, total_u, src_dict in category_stats:
    lines.append(f"#### {cat}")
    lines.append(f"- total unique: {total_u}")
    lines.append(f"- unique by source:")
    for src, cnt in src_dict.items():
        lines.append(f"  - {src}: {cnt} ({cnt/total_u*100:.2f}%)")
    lines.append("")

lines += [
    "### distribution of unique ID_SAPIR per Comb_id",
    *[f"- {num} unique ID_SAPIR: {cnt} ({cnt/dist_comb.count()*100:.2f}%)" for num, cnt in dist_comb_counts.items()],
    "",
    "### distribution of unique ID_SAPIR per Act_ID",
    *[f"- {num} unique ID_SAPIR: {cnt} ({cnt/dist_act.count()*100:.2f}%)" for num, cnt in dist_act_counts.items()],
    "",
    "### ID_SAPIR filled vs empty",
    f"- **filled:** {sapir_filled} ({sapir_filled/total_rows*100:.2f}%)",
    f"- **empty:** {sapir_empty} ({sapir_empty/total_rows*100:.2f}%)",
    "",
    "### top 7 combinations of NAME_CYR and ID_SAPIR",
    *[f"- **{combo}:** {cnt}" for combo, cnt in combo_series.items()],
    "",
    "### top 5 NAME_MISSING_SAPIR",
    *[f"- **{name}:** {cnt}" for name, cnt in top5_missing.items()],
    "",
    "### top 5 NAME_PARACHR_SAPIR",
    *[f"- **{name}:** {cnt}" for name, cnt in top5_parachr.items()],
    "",
    "### GPS coordinates distribution for missing sapir entries",
    f"- **provided:** {gps_provided} ({gps_provided/gps_df.shape[0]*100:.2f}%)",
    f"- **missing:** {gps_missing} ({gps_missing/gps_df.shape[0]*100:.2f}%)"
]
display(Markdown("\n".join(lines)))

# preparing export dataframe
export_cols = [c for c in df_union.columns if c != "effective_name"]
export_df = df_union[export_cols]
export_df.to_excel(output_file, index=False)
print(f"\nmerged file saved to: {output_file}")

# displaying random sample rows
sample_df = export_df.sample(n=10, random_state=42) if total_rows >= 10 else export_df
display(Markdown("### sample of 10 random merged rows"))
display(sample_df)

processed compl_places_unpivoted_with_sapir_admdiv.xlsx, rows: 4
processed compl_places_unpivoted_with_sapir_countries.xlsx, rows: 37
processed compl_places_unpivoted_with_sapir_district.xlsx, rows: 225
processed compl_places_unpivoted_with_sapir_places.xlsx, rows: 142
processed compl_places_unpivoted_with_sapir_region.xlsx, rows: 523
missing files: []


## summary report
- **total rows:** 931

### rows per file
- compl_places_unpivoted_with_sapir_admdiv.xlsx: 4 (0.43%)
- compl_places_unpivoted_with_sapir_countries.xlsx: 37 (3.97%)
- compl_places_unpivoted_with_sapir_district.xlsx: 225 (24.17%)
- compl_places_unpivoted_with_sapir_places.xlsx: 142 (15.25%)
- compl_places_unpivoted_with_sapir_region.xlsx: 523 (56.18%)

### source column stats
- **description_02_uk:** 377 (40.49%)
- **creator:** 371 (39.85%)
- **Places:** 75 (8.06%)
- **Provenance:** 54 (5.80%)
- **note_uk:** 34 (3.65%)
- **ru_details_mod:** 20 (2.15%)

### stats per category (unique effective names)
#### חלוקה_אדמיניסטרטיבית
- total unique: 1
- unique by source:
  - Provenance: 1 (100.00%)

#### countries
- total unique: 1
- unique by source:
  - Provenance: 1 (100.00%)
  - creator: 1 (100.00%)
  - ru_details_mod: 1 (100.00%)

#### district
- total unique: 52
- unique by source:
  - Places: 46 (88.46%)
  - Provenance: 7 (13.46%)
  - description_02_uk: 3 (5.77%)

#### place
- total unique: 36
- unique by source:
  - Places: 4 (11.11%)
  - description_02_uk: 6 (16.67%)
  - note_uk: 25 (69.44%)
  - ru_details_mod: 1 (2.78%)

#### region
- total unique: 14
- unique by source:
  - Places: 5 (35.71%)
  - creator: 10 (71.43%)
  - description_02_uk: 1 (7.14%)
  - ru_details_mod: 1 (7.14%)

### distribution of unique ID_SAPIR per Comb_id
- 1 unique ID_SAPIR: 84 (49.12%)
- 2 unique ID_SAPIR: 84 (49.12%)
- 3 unique ID_SAPIR: 2 (1.17%)
- 17 unique ID_SAPIR: 1 (0.58%)

### distribution of unique ID_SAPIR per Act_ID
- 1 unique ID_SAPIR: 72 (45.00%)
- 2 unique ID_SAPIR: 84 (52.50%)
- 3 unique ID_SAPIR: 2 (1.25%)
- 4 unique ID_SAPIR: 1 (0.62%)
- 17 unique ID_SAPIR: 1 (0.62%)

### ID_SAPIR filled vs empty
- **filled:** 292 (31.36%)
- **empty:** 639 (68.64%)

### top 7 combinations of NAME_CYR and ID_SAPIR
- **Кременецький повіт 7530489:** 80
- **Вишгородок 5558074:** 75
- **Житомирська область 5703815:** 32
- **Українa 5459048:** 19
- **Харьков 5426482:** 18
- **УРСР 5459048:** 17
- **Миколаївська область 5703971:** 10

### top 5 NAME_MISSING_SAPIR
- **Lviv:** 301
- **Ternopil:** 136
- **Lanivtsi:** 75
- **Khmelnytskyi:** 17
- **Rivne:** 13

### top 5 NAME_PARACHR_SAPIR
- **<>,<>,Lwow,Poland:** 299
- **<>,<>,Tarnopol,Poland:** 136
- **<>,Łuck,Wolyn,Poland:** 8
- **<>,Rowne,Wolyn,Poland:** 6
- **<>,<>,Wolyn,Poland:** 3

### GPS coordinates distribution for missing sapir entries
- **provided:** 69 (97.18%)
- **missing:** 2 (2.82%)


merged file saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_unpivoted_with_sapir_unified.xlsx


### sample of 10 random merged rows

,CATEGORY_SAPIR,SOURCE,NAME_CYR,NAME_NORM_CYR,TYPE_NORM_CYR,NAME_SAPIR,FULL_SAPIR,NAME_MISSING_SAPIR,NAME_PARACHR_SAPIR,Comb_id,Act_ID,Sign,ID_SAPIR,ID_PARACHR_SAPIR,MISS_CENTER_GPS,MISS_REF1,MISS_REF2
828,region,description_02_uk,Тернопільська область,Тернопіль,область,NaN,NaN,Ternopil,"<>,<>,Tarnopol,Poland",14384469_3125,6974,4_R-0274_01_00031,,5703951,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,
70,district,description_02_uk,Кременецький повіт,Кременець,повіт,Krzemieniec,"<>,Krzemieniec,Wolyn,Poland",NaN,NaN,14384469_2456,6226,4_0484_01_00129,7530489,,NaN,NaN,
630,region,creator,Львівська область,Львов,область,NaN,NaN,Lviv,"<>,<>,Lwow,Poland",13439437_298,195,1_0005_01_00196,,5703942,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,
506,region,creator,Львівська область,Львов,область,NaN,NaN,Lviv,"<>,<>,Lwow,Poland",13439437_174,82,1_0005_01_00083,,5703942,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,
842,region,description_02_uk,Тернопільська область,Тернопіль,область,NaN,NaN,Ternopil,"<>,<>,Tarnopol,Poland",14384469_3139,6988,4_R-0274_01_00045,,5703951,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,
96,district,description_02_uk,Кременецький повіт,Кременець,повіт,Krzemieniec,"<>,Krzemieniec,Wolyn,Poland",NaN,NaN,14384469_2482,6252,4_0484_01_00155,7530489,,NaN,NaN,
465,region,creator,Львівська область,Львов,область,NaN,NaN,Lviv,"<>,<>,Lwow,Poland",13439437_133,missing,missing,,5703942,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,
86,district,description_02_uk,Кременецький повіт,Кременець,повіт,Krzemieniec,"<>,Krzemieniec,Wolyn,Poland",NaN,NaN,14384469_2472,6242,4_0484_01_00145,7530489,,NaN,NaN,
530,region,creator,Львівська область,Львов,область,NaN,NaN,Lviv,"<>,<>,Lwow,Poland",13439437_198,106,1_0005_01_00107,,5703942,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,
350,place,description_02_uk,Вишгородок,,,Vyshgorodok,"Vyshgorodok,Krzemieniec,Wolyn,Poland",NaN,,14384469_2493,6263,4_0484_01_00166,5558074,,NaN,,


In [ ]:
# @title missing tables structure report
# путь к папке с файлами
file_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"

# спискок файлов для обработки
files = [
    "compl_places_missing_in_sapir_administrative_division.xlsx",
    "compl_places_missing_in_sapir_district.xlsx",
    "compl_places_missing_in_sapir_places.xlsx",
    "compl_places_missing_in_sapir_region.xlsx",
]

# собираем названия колонок
cols_dict = {}
for fname in files:
    df = pd.read_excel(os.path.join(file_dir, fname), nrows=0)
    cols_dict[fname] = list(df.columns)

# определяем максимальную длину списка колонок
max_len = max(len(cols) for cols in cols_dict.values())

# готовим словарь для датафрейма, дополняя списки пустыми строками
export_data = {
    fname: cols + [""] * (max_len - len(cols))
    for fname, cols in cols_dict.items()
}

# создаём и сохраняем итоговый датафрейм
df_export = pd.DataFrame(export_data)
output_path = os.path.join(file_dir, "missing_columns_overview.xlsx")
df_export.to_excel(output_path, index=False)
print(f"columns overview saved to: {output_path}")


columns overview saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/missing_columns_overview.xlsx


In [ ]:
#@title MERGING MISSING

# define base directory for files
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS/COMPL_PLACES_FROM_INCOMINGS"

# read the coordination (union) mapping file
coord_file = os.path.join(base_dir, "compl_missing_union_table.xlsx")
df_mapping = pd.read_excel(coord_file)

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# define final union columns (all columns except 'FILE_NAME')
final_columns = [col for col in df_mapping.columns if col != "FILE_NAME"]

# prepare logging structures
df_list = []
file_row_counts = {}
missing_files = []
missing_columns = {fn: [] for fn in df_mapping["FILE_NAME"]}

# iterate over each mapping row (each corresponds to one source file)
for idx, mapping_row in df_mapping.iterrows():
    file_name = mapping_row["FILE_NAME"]
    category_fixed = mapping_row["CATEGORY_SAPIR"] if pd.notnull(mapping_row["CATEGORY_SAPIR"]) else ""
    file_path = os.path.join(base_dir, file_name)
    print(f"processing file: {file_name}")
    try:
        df_source = pd.read_excel(file_path)
    except Exception:
        missing_files.append(file_name)
        print(f"file not found: {file_name}")
        continue

    df_new = pd.DataFrame(index=df_source.index, columns=final_columns)
    for col in final_columns:
        if col == "CATEGORY_SAPIR":
            df_new[col] = category_fixed
        else:
            mapping_val = mapping_row[col] if pd.notnull(mapping_row[col]) else ""
            if not mapping_val:
                df_new[col] = ""
            elif mapping_val in df_source.columns:
                df_new[col] = df_source[mapping_val]
            else:
                df_new[col] = ""
                missing_columns[file_name].append(mapping_val)
                print(f"{file_name}: column not found {mapping_val}")

    file_row_counts[file_name] = len(df_new)
    print(f"finished processing file: {file_name} with {len(df_new)} rows")
    df_list.append(df_new)

# concatenate all processed dataframes into one final union dataframe
if df_list:
    df_union = pd.concat(df_list, ignore_index=True)
else:
    df_union = pd.DataFrame(columns=final_columns)

# normalize ID_PARACHR_SAPIR
if "ID_PARACHR_SAPIR" in df_union.columns:
    df_union["ID_PARACHR_SAPIR"] = df_union["ID_PARACHR_SAPIR"].apply(normalize_id)

# convert id columns to string without decimals
id_cols = ["comb_id", "Act_ID", "Sign"]
def convert_id_cell(cell):
    if pd.isna(cell) or str(cell).strip() == "":
        return ""
    parts = [p.strip() for p in str(cell).split(",") if p.strip() != ""]
    converted = []
    for part in parts:
        try:
            num = int(float(part))
            converted.append(str(num))
        except:
            converted.append(part)
    return ", ".join(converted)

for col in id_cols:
    if col in df_union.columns:
        df_union[col] = df_union[col].apply(convert_id_cell)

# ---------------------- generating statistical output ----------------------

total_rows = len(df_union)

lines = [
    "## statistical output",
    f"- total rows: {total_rows}",
    "",
    "### rows per category",
]
for cat, cnt in df_union["CATEGORY_SAPIR"].value_counts().items():
    lines.append(f"- {cat}: {cnt} ({cnt/total_rows*100:.2f}%)")

lines += ["", "### distribution by source"]
for src, cnt in df_union["SOURCE"].value_counts().items():
    lines.append(f"- {src}: {cnt} ({cnt/total_rows*100:.2f}%)")

# top 10 name_cyr per category
lines += ["", "### top 10 name_cyr per category"]
df_union["Comb_id_count"] = df_union["Comb_id"].apply(lambda x: len([p for p in str(x).split(",") if p.strip() != ""]) if pd.notna(x) else 0)
grouped = df_union.groupby(["CATEGORY_SAPIR", "NAME_CYR"])["Comb_id_count"].sum().reset_index()
for cat in df_union["CATEGORY_SAPIR"].unique():
    lines.append(f"#### {cat}")
    top10 = grouped[grouped["CATEGORY_SAPIR"]==cat].sort_values("Comb_id_count", ascending=False).head(10)
    for _, row in top10.iterrows():
        lines.append(f"- {row['NAME_CYR']}: {row['Comb_id_count']}")
    lines.append("")

# top 5 name_parachr_sapir
lines += ["", "### top 5 NAME_PARACHR_SAPIR"]
top5_para = df_union[df_union["NAME_PARACHR_SAPIR"]!="" ]["NAME_PARACHR_SAPIR"].value_counts().head(5)
for name, cnt in top5_para.items():
    lines.append(f"- {name}: {cnt}")

# miss_center_gps stats
gps_full = (df_union["MISS_CENTER_GPS"]!="---").sum()
gps_empty = (df_union["MISS_CENTER_GPS"]=="---").sum()
lines += ["", "### MISS_CENTER_GPS stats"]
lines.append(f"- found: {gps_full} ({gps_full/total_rows*100:.2f}%)")
lines.append(f"- missing: {gps_empty} ({gps_empty/total_rows*100:.2f}%)")

# miss_ref1 stats
ref_full = (df_union["MISS_REF1"]!="---").sum()
ref_empty = (df_union["MISS_REF1"]=="---").sum()
lines += ["", "### MISS_REF1 stats"]
lines.append(f"- found: {ref_full} ({ref_full/total_rows*100:.2f}%)")
lines.append(f"- missing: {ref_empty} ({ref_empty/total_rows*100:.2f}%)")

# parachronistic stats for district and region
lines += ["", "### parachronistic stats"]
for cat in ["district", "region"]:
    df_cat = df_union[df_union["CATEGORY_SAPIR"]==cat]
    non_empty = df_cat[df_cat["NAME_PARACHR_SAPIR"].fillna("")!="" ].shape[0]
    lines.append(f"- {cat}: {non_empty}/{len(df_cat)} ({non_empty/len(df_cat)*100:.2f}%)")

# missing files and columns
lines += ["", "### missing files",]
for fn in missing_files:
    lines.append(f"- {fn}")
lines += ["", "### missing columns per file"]
for fn, cols in missing_columns.items():
    if cols:
        lines.append(f"- {fn}: {cols}")

display(Markdown("\n".join(lines)))

# export merged dataframe
export_cols = [c for c in final_columns]
export_df = df_union[export_cols]
output_file = os.path.join(base_dir, "compl_places_missing_in_sapir_unified.xlsx")
export_df.to_excel(output_file, index=False)

print(f"merged file saved to: {output_file}")

# display sample of 10 rows
sample_df = export_df.sample(n=10, random_state=42) if len(export_df)>=10 else export_df
display(sample_df)


processing file: compl_places_missing_in_sapir_administrative_division.xlsx
finished processing file: compl_places_missing_in_sapir_administrative_division.xlsx with 4 rows
processing file: compl_places_missing_in_sapir_district.xlsx
finished processing file: compl_places_missing_in_sapir_district.xlsx with 134 rows
processing file: compl_places_missing_in_sapir_places.xlsx
finished processing file: compl_places_missing_in_sapir_places.xlsx with 9 rows
processing file: compl_places_missing_in_sapir_region.xlsx
finished processing file: compl_places_missing_in_sapir_region.xlsx with 469 rows


## statistical output
- total rows: 616

### rows per category
- region: 469 (76.14%)
- district: 134 (21.75%)
- place: 9 (1.46%)
- חלוקה_אדמיניסטרטיבית: 4 (0.65%)

### distribution by source
- creator: 317 (51.46%)
- description_02_uk: 214 (34.74%)
- Places: 63 (10.23%)
- Provenance: 14 (2.27%)
- note_uk: 8 (1.30%)

### top 10 name_cyr per category
#### חלוקה_אדמיניסטרטיבית
- Львівський евакуаційний район: 4

#### district
- Лановецький район: 75
- Рівненський район: 5
- Винниковский район: 2
- Ковельский район: 2
- Кременецький район: 2
- Дубнівський район: 2
- Львовско-Сельский район: 2
- Вашковский район: 1
- Березновский район: 1
- Вижницкий район: 1

#### place
- Капустинці: 1
- Кобилля: 1
- Колярі: 1
- Малий Глибочок: 1
- Ободівка: 1
- Рохманів: 1
- Сухівці: 1
- Чагарі-Збаразькі: 1
- Шили: 1

#### region
- Львівська область: 294
- Тернопільська область: 136
- Хмельницька область: 17
- Ровенская область: 7
- Львовская область: 5
- Черкаська область: 4
- Волынская область: 3
- Івано-Франківська область: 1
- Дрогобычская область: 1
- Луганська область: 1


### top 5 NAME_PARACHR_SAPIR
- <>,<>,Lwow,Poland: 299
- <>,<>,Tarnopol,Poland: 136
- <>,Rowne,Wolyn,Poland: 6
- <>,<>,Wolyn,Poland: 3
- <>,Lwow,Lwow,Poland: 2

### MISS_CENTER_GPS stats
- found: 614 (99.68%)
- missing: 2 (0.32%)

### MISS_REF1 stats
- found: 616 (100.00%)
- missing: 0 (0.00%)

### parachronistic stats
- district: 21/134 (15.67%)
- region: 438/469 (93.39%)

### missing files

### missing columns per file

merged file saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/COMPL_PLACES_FROM_INCOMINGS/compl_places_missing_in_sapir_unified.xlsx


,CATEGORY_SAPIR,Comb_id,Act_ID,Sign,SOURCE,NAME_CYR,NAME_NORM_CYR,TYPE_NORM_CYR,NAME_ENG,NAME_NORM_ENG,TYPE_NORM_ENG,NAME_PARACHR_SAPIR,MISS_CENTER_GPS,MISS_REF1,ID_PARACHR_SAPIR
78,district,14384469_2503,6273,404840100176,description_02_uk,Лановецький район,Ланівці,район,,Lanivtsi,rayon,NaN,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...,
208,region,13439437_137,53,100050100054,creator,Львівська область,Львов,область,,Lviv,oblast,"<>,<>,Lwow,Poland","49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,5703942
570,region,14384469_3173,7022,4_R-0274_01_00079,description_02_uk,Тернопільська область,Тернопіль,область,,Ternopil,oblast,"<>,<>,Tarnopol,Poland","49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,5703951
181,region,13439437_110,missing,missing,creator,Львівська область,Львов,область,,Lviv,oblast,"<>,<>,Lwow,Poland","49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,5703942
101,district,14384469_2526,6296,404840100199,description_02_uk,Лановецький район,Ланівці,район,,Lanivtsi,rayon,NaN,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...,
158,region,13439437_87,missing,missing,creator,Львівська область,Львов,область,,Lviv,oblast,"<>,<>,Lwow,Poland","49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,5703942
55,district,14384469_2480,6250,404840100153,description_02_uk,Лановецький район,Ланівці,район,,Lanivtsi,rayon,NaN,"49.87029497077897, 26.092374707679674",https://uk.wikipedia.org/wiki/%D0%9B%D0%B0%D0%...,
436,region,13439437_365,257,100050100258,creator,Львівська область,Львов,область,,Lviv,oblast,"<>,<>,Lwow,Poland","49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,5703942
135,district,13569102_15,1512,332295000007,Places,Шацкий район,Шацк,район,,Shatsk,rayon,NaN,"51.497793686969075, 23.93153500198429",https://uk.wikipedia.org/wiki/%D0%A8%D0%B0%D1%...,
218,region,13439437_147,62,100050100063,creator,Львівська область,Львов,область,,Lviv,oblast,"<>,<>,Lwow,Poland","49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,5703942


In [ ]:
#@title MERGING ALL UNPIVOTED

# define root directory and subdirectories
root_dir      = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES"
datasets_dir  = os.path.join(root_dir, "DATASETS")
compl_dir     = os.path.join(datasets_dir, "COMPL_PLACES_FROM_INCOMINGS")

# define file paths
input_title_fond = os.path.join(datasets_dir, "places_unpivoted_with_sapir_unified.xlsx")
input_compl      = os.path.join(compl_dir,    "compl_places_unpivoted_with_sapir_unified.xlsx")
mapping_file     = os.path.join(datasets_dir, "t_General_Act_IDs_with_old_ID.xlsx")
output_file      = os.path.join(root_dir,     "all_places_unpivoted_with_sapir_unified.xlsx")

# read inputs
df_title_fond = pd.read_excel(input_title_fond)
df_compl      = pd.read_excel(input_compl)
df_map        = pd.read_excel(mapping_file)

# loading the normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# normalize mapping-table id columns before building the key dict
for col in ["old_id","ID_master","Comb_ID","Act_ID","Sign"]:
    if col in df_map.columns:
        df_map[col] = df_map[col].apply(normalize_id)

# renaming mapping column so suffix _map applies correctly
df_map.rename(columns={'Comb_ID':'Comb_id'}, inplace=True)

# ensuring both dataframes have all final columns
for col in final_columns:
    if col not in df_title_fond.columns:
        df_title_fond[col] = ""
    if col not in df_compl.columns:
        df_compl[col] = ""

# normalizing df_title_fond ID_OLD and df_compl Comb_id prior to merge
df_title_fond["ID_OLD"] = df_title_fond["ID_OLD"].apply(normalize_id)
df_compl["Comb_id"]      = df_compl["Comb_id"].apply(normalize_id)

# mapping for df_title_fond with safe drops
map_tf = df_map[["old_id", "Comb_id", "Act_ID", "Sign"]].rename(
    columns={
        "Comb_id": "map_Comb_id",
        "Act_ID": "map_Act_ID",
        "Sign": "map_Sign"
    }
)
df_title_fond = df_title_fond.merge(
    map_tf,
    how="left",
    left_on="ID_OLD",
    right_on="old_id"
)

# filling and replacing missing values in df_title_fond
df_title_fond["Comb_id"] = df_title_fond["map_Comb_id"] \
    .fillna("missing") \
    .replace("", "missing")
df_title_fond["Act_ID"]  = df_title_fond["map_Act_ID"] \
    .fillna("missing") \
    .replace("", "missing")
df_title_fond["Sign"]    = df_title_fond["map_Sign"] \
    .fillna("missing") \
    .replace("", "missing")

# dropping helper columns safely
df_title_fond.drop(
    columns=["old_id", "map_Comb_id", "map_Act_ID", "map_Sign"],
    inplace=True,
    errors="ignore"
)

# mapping for df_compl with safe drops
map_compl = df_map[["Comb_id", "old_id", "ID_master"]].rename(
    columns={
        "old_id": "map_old_id",
        "ID_master": "map_ID_master"
    }
)
df_compl = df_compl.merge(
    map_compl,
    how="left",
    on="Comb_id"
)

# filling and replacing missing values for df_compl
# both nan and empty strings will become 'missing'
df_compl["ID_OLD"]    = df_compl["map_old_id"] \
    .fillna("missing") \
    .replace("", "missing")
df_compl["ID_MASTER"] = df_compl["map_ID_master"] \
    .fillna("missing") \
    .replace("", "missing")

# dropping helper columns safely
df_compl.drop(
    columns=["map_old_id", "map_ID_master"],
    inplace=True,
    errors="ignore"
)

# printing mapping coverage statistics after cleaning
total_compl = len(df_compl)
for col in ["ID_OLD", "ID_MASTER"]:
    non_missing = (df_compl[col] != "missing").sum()
    missing     = (df_compl[col] == "missing").sum()
    print(f"df_compl {col}: {non_missing}/{total_compl} ({non_missing/total_compl:.2%}) mapped, missing {missing}/{total_compl} ({missing/total_compl:.2%})")

# printing df_title_fond mapping stats
print()
total_tf = len(df_title_fond)
for col in ["Comb_id","Act_ID","Sign"]:
    non_missing = (df_title_fond[col] != "missing").sum()
    missing     = (df_title_fond[col] == "missing").sum()
    print(f"df_title_fond {col}: {non_missing}/{total_tf} ({non_missing/total_tf:.2%}) mapped, missing {missing}/{total_tf} ({missing/total_tf:.2%})")

# concatenating
df_union = pd.concat(
    [df_title_fond[final_columns], df_compl[final_columns]],
    ignore_index=True
)

# normalizing all id columns in df_union to remove .0 etc
id_cols_to_norm = [
    "ID_OLD","ID_MASTER","Comb_id","Act_ID","Sign",
    "ID_SAPIR","ID_PARACHR_SAPIR","ID_SAPIR_RELATED"
]
for col in id_cols_to_norm:
    if col in df_union.columns:
        df_union[col] = df_union[col].apply(normalize_id)

# ---------------------- generating statistical output ----------------------

total_rows = len(df_union)

# prepare markdown lines
lines = [
    "## summary report",
    f"- **total rows:** {total_rows}",
    "",
    "### rows per file",
]
for fn, cnt in {"title_fond": len(df_title_fond), "compl": len(df_compl)}.items():
    lines.append(f"- {fn}: {cnt} ({cnt/total_rows*100:.2f}%)")

lines += ["", "### distribution by source"]
for src, cnt in df_union["SOURCE"].value_counts().items():
    lines.append(f"- {src}: {cnt} ({cnt/total_rows*100:.2f}%)")

lines += ["", "### stats per category"]
for cat in df_union["CATEGORY_SAPIR"].unique():
    df_cat = df_union[df_union["CATEGORY_SAPIR"] == cat]
    total_unique = df_cat["NAME_SAPIR"].where(
        df_cat["NAME_SAPIR"].fillna("") != "",
        df_cat["NAME_MISSING_SAPIR"]
    ).nunique()
    lines.append(f"#### {cat}")
    lines.append(f"- unique effective names: {total_unique}")
    src_unique = df_cat.groupby("SOURCE")[["NAME_SAPIR","NAME_MISSING_SAPIR"]].nunique().sum(axis=1)
    for src, u in src_unique.items():
        lines.append(f"  - {src}: {u} ({u/total_unique*100:.2f}%)")
    lines.append("")

lines += ["", "### distribution of unique ID_SAPIR per Comb_id"]
dist_comb = df_union[df_union["ID_SAPIR"]!=""].groupby("Comb_id")["ID_SAPIR"].nunique()
for num, cnt in dist_comb.value_counts().sort_index().items():
    lines.append(f"- {num} unique ID_SAPIR: {cnt} ({cnt/dist_comb.count()*100:.2f}%)")

lines += ["", "### distribution of unique ID_SAPIR per Act_ID"]
dist_act = df_union[
    (df_union["Act_ID"]!="") & (df_union["ID_SAPIR"]!="")
].groupby("Act_ID")["ID_SAPIR"].nunique()
for num, cnt in dist_act.value_counts().sort_index().items():
    lines.append(f"- {num} unique ID_SAPIR: {cnt} ({cnt/dist_act.count()*100:.2f}%)")

lines += ["", "### ID_SAPIR filled vs empty"]
sapir_filled = (df_union["ID_SAPIR"]!="").sum()
sapir_empty  = (df_union["ID_SAPIR"]=="").sum()
lines.append(f"- **filled:** {sapir_filled} ({sapir_filled/total_rows*100:.2f}%)")
lines.append(f"- **empty:** {sapir_empty} ({sapir_empty/total_rows*100:.2f}%)")

lines += ["", "### top 50 existing in sapir"]
combo = (
    df_union[(df_union["NAME_CYR"]!="") & (df_union["ID_SAPIR"]!="")]
      .assign(key=lambda x: x["NAME_CYR"] + " " + x["ID_SAPIR"])
      ["key"]
      .value_counts()
      .head(50)
)
for name, cnt in combo.items():
    lines.append(f"- {name}: {cnt}")

lines += ["", "### top 30 missing in sapir"]
mask = (
    (df_union["NAME_MISSING_SAPIR"].fillna("").str.strip() != "") &
    (df_union["NAME_PARACHR_SAPIR"].fillna("").str.strip() == "")
)
top30 = (
    df_union.loc[mask, "NAME_MISSING_SAPIR"]
            .value_counts()
            .head(30)
)
for name, cnt in top30.items():
    lines.append(f"- {name}: {cnt}")

lines += ["", "### top 10 existing, but parachronistic in sapir"]
top10 = df_union[df_union["NAME_PARACHR_SAPIR"]!=""]["NAME_PARACHR_SAPIR"].value_counts().head(10)
for name, cnt in top10.items():
    lines.append(f"- {name}: {cnt}")

# GPS stats
gps_full = (df_union["MISS_CENTER_GPS"]!="---").sum()
gps_empty = (df_union["MISS_CENTER_GPS"]=="---").sum()
lines += ["", "### MISS_CENTER_GPS stats"]
lines.append(f"- found: {gps_full} ({gps_full/total_rows*100:.2f}%)")
lines.append(f"- missing: {gps_empty} ({gps_empty/total_rows*100:.2f}%)")

# REF1 stats
ref_full = (df_union["MISS_REF1"]!="---").sum()
ref_empty = (df_union["MISS_REF1"]=="---").sum()
lines += ["", "### MISS_REF1 stats"]
lines.append(f"- found: {ref_full} ({ref_full/total_rows*100:.2f}%)")
lines.append(f"- missing: {ref_empty} ({ref_empty/total_rows*100:.2f}%)")

# missing columns per file
# (assuming missing_columns dict built earlier if needed)

display(Markdown("\n".join(lines)))

# export
export_df = df_union[final_columns]
export_df.to_excel(output_file, index=False)
print(f"merged file saved to: {output_file}")

# sample 30 rows
sample_df = export_df.sample(n=30, random_state=42) if len(export_df)>=30 else export_df
display(sample_df)


df_compl ID_OLD: 20/931 (2.15%) mapped, missing 911/931 (97.85%)
df_compl ID_MASTER: 762/931 (81.85%) mapped, missing 169/931 (18.15%)

df_title_fond Comb_id: 18826/24234 (77.68%) mapped, missing 5408/24234 (22.32%)
df_title_fond Act_ID: 18826/24234 (77.68%) mapped, missing 5408/24234 (22.32%)
df_title_fond Sign: 18826/24234 (77.68%) mapped, missing 5408/24234 (22.32%)


## summary report
- **total rows:** 25165

### rows per file
- title_fond: 24234 (96.30%)
- compl: 931 (3.70%)

### distribution by source
- fond: 14075 (55.93%)
- title: 10159 (40.37%)
- description_02_uk: 377 (1.50%)
- creator: 371 (1.47%)
- Places: 75 (0.30%)
- Provenance: 54 (0.21%)
- note_uk: 34 (0.14%)
- ru_details_mod: 20 (0.08%)

### stats per category
#### חלוקה_אדמיניסטרטיבית
- unique effective names: 14
  - Provenance: 1 (7.14%)
  - title: 13 (92.86%)

#### countries
- unique effective names: 24
  - Provenance: 1 (4.17%)
  - creator: 1 (4.17%)
  - fond: 2 (8.33%)
  - ru_details_mod: 1 (4.17%)
  - title: 24 (100.00%)

#### district
- unique effective names: 187
  - Places: 46 (24.60%)
  - Provenance: 7 (3.74%)
  - description_02_uk: 3 (1.60%)
  - fond: 19 (10.16%)
  - title: 130 (69.52%)

#### place
- unique effective names: 763
  - Places: 4 (0.52%)
  - description_02_uk: 6 (0.79%)
  - fond: 40 (5.24%)
  - note_uk: 25 (3.28%)
  - ru_details_mod: 1 (0.13%)
  - title: 740 (96.99%)

#### region
- unique effective names: 36
  - Places: 5 (13.89%)
  - creator: 10 (27.78%)
  - description_02_uk: 1 (2.78%)
  - fond: 13 (36.11%)
  - ru_details_mod: 1 (2.78%)
  - title: 24 (66.67%)

#### sub-district
- unique effective names: 43
  - fond: 5 (11.63%)
  - title: 41 (95.35%)


### distribution of unique ID_SAPIR per Comb_id
- 1 unique ID_SAPIR: 6953 (77.16%)
- 2 unique ID_SAPIR: 1534 (17.02%)
- 3 unique ID_SAPIR: 326 (3.62%)
- 4 unique ID_SAPIR: 120 (1.33%)
- 5 unique ID_SAPIR: 35 (0.39%)
- 6 unique ID_SAPIR: 7 (0.08%)
- 7 unique ID_SAPIR: 4 (0.04%)
- 8 unique ID_SAPIR: 3 (0.03%)
- 9 unique ID_SAPIR: 2 (0.02%)
- 10 unique ID_SAPIR: 2 (0.02%)
- 12 unique ID_SAPIR: 11 (0.12%)
- 14 unique ID_SAPIR: 2 (0.02%)
- 15 unique ID_SAPIR: 1 (0.01%)
- 16 unique ID_SAPIR: 3 (0.03%)
- 17 unique ID_SAPIR: 2 (0.02%)
- 18 unique ID_SAPIR: 2 (0.02%)
- 21 unique ID_SAPIR: 2 (0.02%)
- 23 unique ID_SAPIR: 1 (0.01%)
- 801 unique ID_SAPIR: 1 (0.01%)

### distribution of unique ID_SAPIR per Act_ID
- 1 unique ID_SAPIR: 6941 (77.13%)
- 2 unique ID_SAPIR: 1534 (17.05%)
- 3 unique ID_SAPIR: 326 (3.62%)
- 4 unique ID_SAPIR: 120 (1.33%)
- 5 unique ID_SAPIR: 35 (0.39%)
- 6 unique ID_SAPIR: 7 (0.08%)
- 7 unique ID_SAPIR: 4 (0.04%)
- 8 unique ID_SAPIR: 3 (0.03%)
- 9 unique ID_SAPIR: 2 (0.02%)
- 10 unique ID_SAPIR: 2 (0.02%)
- 12 unique ID_SAPIR: 11 (0.12%)
- 14 unique ID_SAPIR: 2 (0.02%)
- 15 unique ID_SAPIR: 1 (0.01%)
- 16 unique ID_SAPIR: 3 (0.03%)
- 17 unique ID_SAPIR: 2 (0.02%)
- 18 unique ID_SAPIR: 2 (0.02%)
- 21 unique ID_SAPIR: 2 (0.02%)
- 23 unique ID_SAPIR: 1 (0.01%)
- 804 unique ID_SAPIR: 1 (0.01%)

### ID_SAPIR filled vs empty
- **filled:** 20009 (79.51%)
- **empty:** 5156 (20.49%)

### top 50 existing in sapir
- Волынское воеводство 5703791: 1853
- Калінінський район 5703891: 1579
- Херсон 5426489: 1404
- Тернопіль 5547367: 1122
- Калінінське 5500289: 1027
- Калініндорфський район 5703891: 902
- Луцьк 5511940: 857
- Кременець 5507084: 813
- Херсонський округ 5703930: 564
- Кременецький повіт 7530489: 480
- Польща 5696006: 419
- Николаев 5520607: 389
- Николаевская область 5703971: 388
- Волинь 5703791: 386
- Луцк 5511940: 236
- Бережани 5476417: 218
- Тернопільський округ 5719882: 183
- Тернопільське воєводство 5719882: 181
- Тернопільський повіт 5719882: 178
- Волинське воєводство 5703791: 175
- Україна 5459048: 163
- Львове 5512611: 131
- Чортків 5483439: 114
- Вишгородок 5558074: 97
- Скалатський повіт 6472998: 95
- Золочівський повіт 7557962: 95
- Бобровий Кут 5473750: 95
- Вищегрудек 5444910: 93
- Варшава 5555992: 89
- Копичинці 5504497: 88
- Підволочиськ 13069516: 87
- Ровно 5534079: 85
- Бучацький повіт 7526963: 75
- Палестина 5700529: 70
- Броди 5422172: 69
- Скалат 5440532: 68
- Бродівський повіт 7526306: 68
- Чехословацька республіка 5706532: 67
- Чортківський повіт 6473110: 62
- Херсонський повіт 5703930: 59
- Бучач 5476767: 59
- Бережанський повіт 7526389: 59
- СРСР 5706238: 53
- Борщівський повіт 6822483: 52
- Підгайці 5528070: 50
- Золочів 5561190: 50
- Збаразький повіт 6330387: 50
- Теребовлянський повіт 6822508: 49
- Бериславський район 7561983: 48
- Львiв 5512624: 47

### top 30 missing in sapir
- Kherson okruha: 300
- Ozeriany: 225
- Kremenets raion: 153
- Bobrovyi Kut rural council: 146
- Kherson oblast: 135
- Snihurivka raion: 133
- Khrushchove: 96
- Kirove: 90
- Lanivtsi: 75
- Kalininske settlement council: 56
- Mykhailivka: 55
- Novopoltavka: 52
- Chortkiv OUN supra-raion: 46
- Kamianka-Strumylovska powiat: 42
- Ivanovka: 41
- Kalynivka: 40
- Novohrednieve rural council: 34
- Kalininske rural council: 32
- Volitsa: 32
- Pervomaisk: 32
- Voroshilove: 31
- Leninske rural council: 31
- Khrushchove rural council: 30
- Voroshilove rural council: 30
- Lanivtsi raion: 27
- Chkalove rural council: 27
- Kuibysheve rural council: 27
- Dobrovody volost: 23
- Borki: 21
- Stavky: 20

### top 10 existing, but parachronistic in sapir
- <>,<>,Carpathian Ruthenia,Czechoslovakia: 689
- <>,<>,Tarnopol,Poland: 636
- <>,<>,Lwow,Poland: 549
- <>,<>,Odessa,Ukraine (USSR): 148
- <>,<>,Wolyn,Poland: 105
- <>,Zbaraz,Tarnopol,Poland: 37
- <>,Łuck,Wolyn,Poland: 33
- <>,Zaleszczyki,Tarnopol,Poland: 20
- <>,Podhajce,Tarnopol,Poland: 20
- <>,Kopyczynce,Tarnopol,Poland: 14

### MISS_CENTER_GPS stats
- found: 24508 (97.39%)
- missing: 657 (2.61%)

### MISS_REF1 stats
- found: 24949 (99.14%)
- missing: 216 (0.86%)

merged file saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/all_places_unpivoted_with_sapir_unified.xlsx


,CATEGORY_SAPIR,SOURCE,NAME_CYR,NAME_NORM_CYR,TYPE_NORM_CYR,NAME_SAPIR,FULL_SAPIR,NAME_MISSING_SAPIR,NAME_PARACHR_SAPIR,ID_OLD,...,Act_ID,Sign,ID_SAPIR,ID_PARACHR_SAPIR,ID_SAPIR_RELATED,CODE_SAPIR,MISS_CENTER_GPS,MISS_REF1,MISS_REF2,MISS_REF3
24035,sub-district,fond,Шоломалейхімська сільрада,Шолом Алейхем,сільрада,SHOLOM ALEYKHEM,NaN,NaN,NaN,15060,...,15006,7_R-2479_01_00014,,,"5435440, 5464965, 5707013, 6089197, 6089208, 7...",T9184734,NaN,NaN,NaN,NaN
20940,region,fond,Закарпатская область,Закарпатье,область,NaN,NaN,Transcarpathian oblast,"<>,<>,Carpathian Ruthenia,Czechoslovakia",7427,...,7912,5_P-3223_02_00111,,5720537,,NaN,"48.62242699013335, 22.287959651054354",https://uk.wikipedia.org/wiki/%D0%97%D0%B0%D0%...,NaN,NaN
18027,place,title,Первомайськ,NaN,NaN,NaN,"Pervomaysk,Imeni Kaganovicha,Voroshilovgrad,Uk...",Pervomaisk,NaN,15099,...,15044,7_R-2479_01_00052,10414301,,,NaN,---,NaN,NaN,NaN
14682,place,title,Чортків,NaN,NaN,Chortkiv,"Chortkiv,Czortkow,Tarnopol,Poland",NaN,NaN,5136,...,missing,missing,5483439,,,NaN,NaN,NaN,NaN,NaN
22060,region,fond,Херсонська округа,Херсон,округа,NaN,NaN,Kherson okruha,NaN,18584,...,11496,7_R-0737_01_00016,,,,NaN,"46.654020677463755, 32.599544743751984",https://uk.wikipedia.org/wiki/%D0%A5%D0%B5%D1%...,NaN,NaN
1675,district,title,Кременецький повіт,Кременець,повіт,Krzemieniec,"<>,Krzemieniec,Wolyn,Poland",NaN,NaN,6302,...,missing,missing,7530489,,,NaN,NaN,NaN,NaN,NaN
12620,place,fond,Калінінське,NaN,NaN,Kalininske,"Kalininske,Kalinindorf,Nikolayev,Ukraine (USSR)",NaN,NaN,14630,...,14130,7_R-2436_02_00040,5500289,,,NaN,NaN,NaN,NaN,NaN
3000,district,title,Чортківський район,Чортків,район,NaN,NaN,Chortkiv raion,NaN,10285,...,6508,4_R-0008_01_01021,,,,NaN,"49.01376773097081, 25.800348180414147",https://uk.wikipedia.org/wiki/%D0%A7%D0%BE%D1%...,NaN,NaN
312,countries,fond,Польща,NaN,NaN,Poland,"<>,<>,<>,Poland",NaN,NaN,7920,...,2454,4_0015_01_00252,5696006,,,NaN,NaN,NaN,NaN,NaN
4275,district,fond,Калініндорфський район,Калініндорф,район,Kalinindorf,"<>,Kalinindorf,Nikolayev,Ukraine (USSR)",NaN,NaN,13185,...,13148,7_R-2422_01_00005,5703891,,,NaN,NaN,NaN,NaN,NaN


In [ ]:
#@title MERGING ALL MISSING

# defining root and dataset directories
root_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES"
datasets_dir = os.path.join(root_dir, "DATASETS")
compl_dir = os.path.join(datasets_dir, "COMPL_PLACES_FROM_INCOMINGS")

# defining input and output file paths
input_title_fond_missing = os.path.join(datasets_dir, "places_missing_in_sapir_unified.xlsx")
input_compl_missing = os.path.join(compl_dir, "compl_places_missing_in_sapir_unified.xlsx")
mapping_file        = os.path.join(datasets_dir, "t_General_Act_IDs_with_old_ID.xlsx")
output_file         = os.path.join(root_dir, "all_places_missing_in_sapir_unified.xlsx")

# reading input files
df_title_fond_missing = pd.read_excel(input_title_fond_missing)
df_compl_missing = pd.read_excel(input_compl_missing)
df_map           = pd.read_excel(mapping_file)

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# normalizing mapping table id columns
for col in ["old_id", "ID_master", "Comb_ID", "Act_ID", "Sign"]:
    if col in df_map.columns:
        df_map[col] = df_map[col].apply(normalize_id)

# renaming mapping Comb_ID column for consistency
df_map.rename(columns={"Comb_ID": "Comb_id"}, inplace=True)

# defining final columns in desired order
final_columns = [
    "CATEGORY_SAPIR", "SOURCE", "NAME_CYR", "NAME_NORM_CYR", "TYPE_NORM_CYR",
    "NAME_ENG", "NAME_NORM_ENG", "TYPE_NORM_ENG", "NAME_PARACHR_SAPIR",
    "LACKING_DEDICATED_SAPIR", "ID_OLD", "ID_MASTER", "Comb_id", "Act_ID",
    "Sign", "ID_PARACHR_SAPIR", "ID_RELATED_SAPIR", "CODE_SAPIR",
    "MISS_CENTER_GPS", "MISS_REF1", "MISS_REF2", "MISS_REF3"
]

# ensuring both dataframes have all final columns
for col in final_columns:
    if col not in df_title_fond_missing.columns:
        df_title_fond_missing[col] = ""
    if col not in df_compl_missing.columns:
        df_compl_missing[col] = ""

# normalizing key id columns before merge
df_title_fond_missing["ID_OLD"]   = df_title_fond_missing["ID_OLD"].apply(normalize_id)
df_compl_missing["Comb_id"]  = df_compl_missing["Comb_id"].apply(normalize_id)

# mapping for title_missing: linking old_id to comb_id, Act_ID, Sign
map_title = df_map[["old_id","Comb_id","Act_ID","Sign"]].rename(
    columns={"Comb_id":"map_Comb_id","Act_ID":"map_Act_ID","Sign":"map_Sign"}
)
df_title_fond_missing = df_title_fond_missing.merge(
    map_title,
    how="left",
    left_on="ID_OLD",
    right_on="old_id"
)

df_title_fond_missing["Comb_id"] = df_title_fond_missing["map_Comb_id"].fillna("missing").replace("","missing")
df_title_fond_missing["Act_ID"]  = df_title_fond_missing["map_Act_ID"].fillna("missing").replace("","missing")
df_title_fond_missing["Sign"]    = df_title_fond_missing["map_Sign"].fillna("missing").replace("","missing")

# dropping helper columns from title_missing
df_title_fond_missing.drop(
    columns=["old_id","map_Comb_id","map_Act_ID","map_Sign"],
    inplace=True,
    errors="ignore"
)

# mapping for compl_missing: linking comb_id to ID_OLD and ID_MASTER
map_compl = df_map[["Comb_id","old_id","ID_master"]].rename(
    columns={"old_id":"map_old_id","ID_master":"map_ID_master"}
)
df_compl_missing = df_compl_missing.merge(
    map_compl,
    how="left",
    on="Comb_id"
)

df_compl_missing["ID_OLD"]    = df_compl_missing["map_old_id"].fillna("missing").replace("","missing")
df_compl_missing["ID_MASTER"] = df_compl_missing["map_ID_master"].fillna("missing").replace("","missing")

# dropping helper columns from compl_missing
df_compl_missing.drop(
    columns=["map_old_id","map_ID_master"],
    inplace=True,
    errors="ignore"
)

# printing mapping coverage statistics after cleaning
total_compl = len(df_compl_missing)
for col in ["ID_OLD", "ID_MASTER"]:
    non_missing = (df_compl_missing[col] != "missing").sum()
    missing     = (df_compl_missing[col] == "missing").sum()
    print(f"df_compl_missing {col}: {non_missing}/{total_compl} ({non_missing/total_compl:.2%}) mapped, missing {missing}/{total_compl} ({missing/total_compl:.2%})")

print()
total_tf = len(df_title_fond_missing)
for col in ["Comb_id", "Act_ID", "Sign"]:
    non_missing = (df_title_fond_missing[col] != "missing").sum()
    missing     = (df_title_fond_missing[col] == "missing").sum()
    print(f"df_title_fond_missing {col}: {non_missing}/{total_tf} ({non_missing/total_tf:.2%}) mapped, missing {missing}/{total_tf} ({missing/total_tf:.2%})")

# concatenating title_missing and compl_missing into one dataframe
df_union = pd.concat(
    [df_title_fond_missing[final_columns], df_compl_missing[final_columns]],
    ignore_index=True
)
df_union["ID_MASTER"] = df_union["ID_MASTER"].replace(["", np.nan], "missing")

# normalizing all id-like columns in the combined dataframe
id_cols_to_norm = [
    "ID_OLD","ID_MASTER","Comb_id","Act_ID","Sign",
    "ID_PARACHR_SAPIR","ID_RELATED_SAPIR","CODE_SAPIR"
]
for col in id_cols_to_norm:
    if col in df_union.columns:
        df_union[col] = df_union[col].apply(normalize_id)

# generating statistical output
total_rows = len(df_union)

lines = [
    "## statistical output",
    f"- total rows: {total_rows}",
    "",
    "### rows per category"
]
for cat, cnt in df_union["CATEGORY_SAPIR"].value_counts().items():
    perc = cnt/total_rows*100 if total_rows else 0
    lines.append(f"- {cat}: {cnt} ({perc:.2f}%)")

lines += ["", "### distribution by source"]
for src, cnt in df_union["SOURCE"].value_counts().items():
    perc = cnt/total_rows*100 if total_rows else 0
    lines.append(f"- {src}: {cnt} ({perc:.2f}%)")

lines += ["", "### top 5 name_cyr per category"]
df_union["Comb_id_count"] = df_union["Comb_id"].apply(
    lambda x: len([p for p in str(x).split(",") if p.strip() != ""])
)
grouped = df_union.groupby(["CATEGORY_SAPIR","NAME_CYR"])["Comb_id_count"].sum().reset_index()
for cat in df_union["CATEGORY_SAPIR"].unique():
    lines.append(f"#### {cat}")
    top5 = grouped[grouped["CATEGORY_SAPIR"]==cat] \
               .sort_values("Comb_id_count", ascending=False) \
               .head(5)
    for _, row in top5.iterrows():
        lines.append(f"- {row['NAME_CYR']}: {row['Comb_id_count']}")
    lines.append("")

lines += ["", "### top 10 NAME_PARACHR_SAPIR"]
top10_para = df_union[df_union["NAME_PARACHR_SAPIR"]!=""]["NAME_PARACHR_SAPIR"] \
                .value_counts().head(10)
for name, cnt in top10_para.items():
    lines.append(f"- {name}: {cnt}")

lines += ["", "### MISS_CENTER_GPS stats"]
gps_full  = (df_union["MISS_CENTER_GPS"]!="---").sum()
gps_empty = (df_union["MISS_CENTER_GPS"]=="---").sum()
lines.append(f"- found: {gps_full} ({gps_full/total_rows*100:.2f}%)")
lines.append(f"- missing: {gps_empty} ({gps_empty/total_rows*100:.2f}%)")

lines += ["", "### MISS_REF1 stats"]
ref_full  = (df_union["MISS_REF1"]!="---").sum()
ref_empty = (df_union["MISS_REF1"]=="---").sum()
lines.append(f"- found: {ref_full} ({ref_full/total_rows*100:.2f}%)")
lines.append(f"- missing: {ref_empty} ({ref_empty/total_rows*100:.2f}%)")

lines += ["", "### parachronistic stats"]
for cat in ["district","region"]:
    df_cat = df_union[df_union["CATEGORY_SAPIR"]==cat]
    non_empty = df_cat[df_cat["NAME_PARACHR_SAPIR"].fillna("")!="" ].shape[0]
    total_cat = len(df_cat)
    perc = non_empty/total_cat*100 if total_cat else 0
    lines.append(f"- {cat}: {non_empty}/{total_cat} ({perc:.2f}%)")

display(Markdown("\n".join(lines)))

# exporting the combined dataframe
export_df = df_union[final_columns]
export_df.to_excel(output_file, index=False)
print(f"merged file saved to: {output_file}")

# displaying a random sample of 20 rows for verification
sample_df = export_df.sample(n=20, random_state=42) if len(export_df)>=20 else export_df
display(sample_df)

df_compl_missing ID_OLD: 11/616 (1.79%) mapped, missing 605/616 (98.21%)
df_compl_missing ID_MASTER: 519/616 (84.25%) mapped, missing 97/616 (15.75%)

df_title_fond_missing Comb_id: 122/490 (24.90%) mapped, missing 368/490 (75.10%)
df_title_fond_missing Act_ID: 122/490 (24.90%) mapped, missing 368/490 (75.10%)
df_title_fond_missing Sign: 122/490 (24.90%) mapped, missing 368/490 (75.10%)


## statistical output
- total rows: 1106

### rows per category
- region: 505 (45.66%)
- place: 293 (26.49%)
- district: 232 (20.98%)
- sub-district: 59 (5.33%)
- חלוקה_אדמיניסטרטיבית: 17 (1.54%)

### distribution by source
- title: 466 (42.13%)
- creator: 317 (28.66%)
- description_02_uk: 214 (19.35%)
- Places: 63 (5.70%)
- fond: 24 (2.17%)
- Provenance: 14 (1.27%)
- note_uk: 8 (0.72%)

### top 5 name_cyr per category
#### חלוקה_אדמיניסטרטיבית
- Львівський евакуаційний район: 4
- Бережанська округа ОУН: 1
- Бережанський надрайон ОУН: 1
- Бучацький виборчий округ: 1
- Бучацький надрайон ОУН: 1

#### district
- Лановецький район: 76
- Рівненський район: 5
- Кременецький район: 3
- Дніпровський повіт: 2
- Винниковский район: 2

#### place
- колгосп Авангард Калінінської сільради: 2
- Кобилля: 2
- Великі Кусківці: 2
- Рохманів: 2
- Долина: 2

#### region
- Львівська область: 296
- Тернопільська область: 138
- Хмельницька область: 17
- Ровенская область: 7
- Львовская область: 5

#### sub-district
- Бобровокутська сільська рада: 2
- Шоломалейхімська сільрада: 2
- Базарська сільрада: 1
- Вавилівська сільрада: 1
- Бобровокутська сільрада: 1


### top 10 NAME_PARACHR_SAPIR
- <>,<>,Lwow,Poland: 303
- <>,<>,Tarnopol,Poland: 139
- <>,<>,Wolyn,Poland: 10
- <>,<>,Carpathian Ruthenia,Czechoslovakia: 6
- <>,Rowne,Wolyn,Poland: 6
- <>,Łuck,Wolyn,Poland: 4
- <>,<>,Odessa,Ukraine (USSR): 2
- <>,Dubno,Wolyn,Poland: 2
- <>,Lwow,Lwow,Poland: 2
- <>,Krzemieniec,Wolyn,Poland: 2

### MISS_CENTER_GPS stats
- found: 986 (89.15%)
- missing: 120 (10.85%)

### MISS_REF1 stats
- found: 1085 (98.10%)
- missing: 21 (1.90%)

### parachronistic stats
- district: 45/232 (19.40%)
- region: 463/505 (91.68%)

merged file saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/all_places_missing_in_sapir_unified.xlsx


,CATEGORY_SAPIR,SOURCE,NAME_CYR,NAME_NORM_CYR,TYPE_NORM_CYR,NAME_ENG,NAME_NORM_ENG,TYPE_NORM_ENG,NAME_PARACHR_SAPIR,LACKING_DEDICATED_SAPIR,...,Comb_id,Act_ID,Sign,ID_PARACHR_SAPIR,ID_RELATED_SAPIR,CODE_SAPIR,MISS_CENTER_GPS,MISS_REF1,MISS_REF2,MISS_REF3
323,place,title,Мазури,NaN,NaN,Mazury,NaN,NaN,NaN,NaN,...,missing,missing,missing,,,,---,NaN,NaN,NaN
732,region,creator,Львівська область,Львов,область,NaN,Lviv,oblast,"<>,<>,Lwow,Poland",,...,13439437_171,80,100050100081,5703942,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,,
56,district,title,Почаївський район,Почаїв,район,Pochaiv raion,Pochaiv,raion,NaN,NaN,...,missing,missing,missing,,,,"50.002686682433705, 25.508573749981334",https://ternopedia.te.ua/index.php/%D0%9F%D0%B...,NaN,NaN
693,region,creator,Львівська область,Львов,область,NaN,Lviv,oblast,"<>,<>,Lwow,Poland",,...,13439437_132,49,100050100050,5703942,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,,
485,sub-district,title,Леніндорфська сільрада,Леніндорф,сільрада,NaN,NaN,NaN,NaN,Lenindorf,...,15070965_547,12681,7_R-1479_01_00129,,"5432787, 5509691, 7529673, 8778992, 9653669, 9...",T9334673,NaN,NaN,NaN,NaN
807,region,creator,Львівська область,Львов,область,NaN,Lviv,oblast,"<>,<>,Lwow,Poland",,...,13439437_246,150,100050100151,5703942,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,,
1060,region,description_02_uk,Тернопільська область,Тернопіль,область,NaN,Ternopil,oblast,"<>,<>,Tarnopol,Poland",,...,14384469_3173,7022,4_R-0274_01_00079,5703951,,,"49.55367100422771, 25.604881829034692",https://uk.wikipedia.org/wiki/%D0%A2%D0%B5%D1%...,,
86,district,title,Гороховский район,Горохов,район,Horokhov raion,Horokhov,raion,"<>,Horochow,Wolyn,Poland",NaN,...,12804595_49,16670,9_0002_01_00101,7529609,,,"50.49882078316198, 24.760630380923534",https://uk.wikipedia.org/wiki/%D0%93%D0%BE%D1%...,NaN,NaN
218,place,title,Ново-Чарторийск,NaN,NaN,Novo-Chartoryisk,NaN,NaN,NaN,NaN,...,missing,missing,missing,,,,"51.299870177490206, 25.83426739614368",NaN,NaN,NaN
877,region,creator,Львівська область,Львов,область,NaN,Lviv,oblast,"<>,<>,Lwow,Poland",,...,13439437_316,211,100050100212,5703942,,,"49.84289937554935, 24.002704057787373",https://uk.wikipedia.org/wiki/%D0%9B%D1%8C%D0%...,,


#   ימי שני

*   עבדתי שעה וחצי בבית

*   פעם שעברה שעתיים
*   ולפני זה בכלל לא

In [ ]:
#@title MAPPING_COVERAGE_CHECK

import os
import pandas as pd

# file paths (adjust if needed)
root_dir       = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES"
datasets_dir   = os.path.join(root_dir, "DATASETS")
compl_dir      = os.path.join(datasets_dir, "COMPL_PLACES_FROM_INCOMINGS")
mapping_file   = os.path.join(datasets_dir, "t_General_Act_IDs_with_old_ID.xlsx")
compl_file     = os.path.join(compl_dir,    "compl_places_missing_in_sapir_unified.xlsx")

# load data
df_map   = pd.read_excel(mapping_file)
df_compl = pd.read_excel(compl_file)

# normalize and prepare mapping DataFrame
df_map['comb_norm']    = df_map['Comb_ID'].astype(str).str.strip()
df_map['old_norm']     = df_map['old_id'].astype(str).str.strip()
df_map['master_norm']  = df_map['ID_master'].astype(str).str.strip()

map_df = df_map[['comb_norm', 'old_norm', 'master_norm']].drop_duplicates()

# prepare unique comb_ids from compl data
df_chk = (
    df_compl
    .assign(comb_norm=df_compl['Comb_id'].astype(str).str.strip())
    .drop_duplicates('comb_norm')[['comb_norm']]
    .merge(map_df, on='comb_norm', how='left')
)

total = len(df_chk)
mapped_old   = df_chk['old_norm'].notna().sum()
mapped_master= df_chk['master_norm'].notna().sum()

print(f"total unique comb_id      : {total}")
print(f"mapped to old_id          : {mapped_old}/{total}")
print(f"mapped to ID_master       : {mapped_master}/{total}")
print("\ncomb_ids missing old_id   :", df_chk.loc[df_chk['old_norm'].isna(), 'comb_norm'].tolist())
print("comb_ids missing master_id:", df_chk.loc[df_chk['master_norm'].isna(), 'comb_norm'].tolist())


total unique comb_id      : 549
mapped to old_id          : 518/549
mapped to ID_master       : 518/549

comb_ids missing old_id   : ['13439437_87', '13439437_101', '13439437_110', '13439437_112', '13439437_118', '13439437_119', '13439437_133', '13439437_146', '13439437_149', '13439437_152', '13439437_157', '13439437_160', '13439437_166', '13439437_168', '13439437_172', '13439437_217', '13439437_228', '13439437_233', '13439437_235', '13439437_264', '13439437_268', '13439437_275', '13439437_277', '13439437_279', '13439437_282', '13439437_296', '13439437_311', '13439437_314', '13439437_321', '13439437_362', '13439437_364']
comb_ids missing master_id: ['13439437_87', '13439437_101', '13439437_110', '13439437_112', '13439437_118', '13439437_119', '13439437_133', '13439437_146', '13439437_149', '13439437_152', '13439437_157', '13439437_160', '13439437_166', '13439437_168', '13439437_172', '13439437_217', '13439437_228', '13439437_233', '13439437_235', '13439437_264', '13439437_268', '134394

In [ ]:
import os
import pandas as pd

# 1) пути к файлам (подкорректируйте при необходимости)
base_dir       = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"
input_title    = os.path.join(base_dir, "places_unpivoted_with_sapir_unified.xlsx")
input_compl    = os.path.join(base_dir, "COMPL_PLACES_FROM_INCOMINGS", "compl_places_unpivoted_with_sapir_unified.xlsx")
mapping_file   = os.path.join(base_dir, "t_General_Act_IDs_with_old_ID.xlsx")

# 2) читаем данные
df_title = pd.read_excel(input_title)
df_compl = pd.read_excel(input_compl)
df_map   = pd.read_excel(mapping_file)

# 3) формируем df_union точно так же, как в основном скрипте
final_columns = [
    "CATEGORY_SAPIR","SOURCE","NAME_CYR","NAME_NORM_CYR","TYPE_NORM_CYR",
    "NAME_SAPIR","FULL_SAPIR","NAME_MISSING_SAPIR","NAME_PARACHR_SAPIR",
    "ID_OLD","ID_MASTER","Comb_id","Act_ID","Sign",
    "ID_SAPIR","ID_PARACHR_SAPIR","ID_SAPIR_RELATED","CODE_SAPIR",
    "MISS_CENTER_GPS","MISS_REF1","MISS_REF2","MISS_REF3"
]
for col in final_columns:
    if col not in df_title.columns: df_title[col] = ""
    if col not in df_compl.columns: df_compl[col] = ""
df_union = pd.concat([df_title[final_columns], df_compl[final_columns]], ignore_index=True)

# 4) короткая debug-проверка ДО normalize/map_all_ids
print("\n>>> DEBUG BEFORE MAPPING <<<\n")

# A) ID_OLD
u_old = set(df_union["ID_OLD"].fillna("").astype(str).unique())
m_old = set(df_map["old_id"].fillna("").astype(str).unique())
print("union.ID_OLD (sample):", sorted(u_old)[:20])
print("mapping.old_id (sample):", sorted(m_old)[:20])
print("IDs in union but not in map.old_id:", sorted(u_old - m_old)[:20])
print("IDs in map.old_id but not in union:", sorted(m_old - u_old)[:20], "\n")

# B) Comb_id
u_comb = set(df_union["Comb_id"].fillna("").astype(str).unique())
m_comb = set(df_map["Comb_ID"].fillna("").astype(str).unique())
print("union.Comb_id (sample):", sorted(u_comb)[:20])
print("mapping.Comb_ID (sample):", sorted(m_comb)[:20])
print("Comb_ids in union but not in map.Comb_ID:", sorted(u_comb - m_comb)[:20])
print("Comb_IDs in map but not in union:", sorted(m_comb - u_comb)[:20], "\n")

# C) строки без соответствий ни по old_id, ни по Comb_ID
bad_mask = ~df_union["ID_OLD"].astype(str).isin(m_old) & ~df_union["Comb_id"].astype(str).isin(m_comb)
bad_examples = df_union.loc[bad_mask, ["ID_OLD","Comb_id"]].drop_duplicates().head(20)
print("пример строк без соответствий:", bad_examples.to_string(index=False))



>>> DEBUG BEFORE MAPPING <<<

union.ID_OLD (sample): ['', '1', '10', '100', '1000', '10000', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '1001', '10010', '10011', '10012', '10013']
mapping.old_id (sample): ['', '1.0', '10.0', '100.0', '1000.0', '10000.0', '10001.0', '10002.0', '10003.0', '10004.0', '10005.0', '10006.0', '10007.0', '10008.0', '10009.0', '1001.0', '10010.0', '10011.0', '10012.0', '10013.0']
IDs in union but not in map.old_id: ['1', '10', '100', '1000', '10000', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '1001', '10010', '10011', '10012', '10013', '10014']
IDs in map.old_id but not in union: ['1.0', '10.0', '100.0', '1000.0', '10000.0', '10001.0', '10002.0', '10003.0', '10004.0', '10005.0', '10006.0', '10007.0', '10008.0', '10009.0', '1001.0', '10010.0', '10011.0', '10012.0', '10013.0', '10014.0'] 

union.Comb_id (sample): ['', '11803151_1', '11803151_15', '11803151_17', '11803151_2', '11803151_20

In [ ]:
#@title GENERATE COMB_ID_MAPPING_FRAGMENT
import os
import pandas as pd

# defining root and data directories
root_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES"
datasets_dir = os.path.join(root_dir, "DATASETS")
compl_dir = os.path.join(datasets_dir, "COMPL_PLACES_FROM_INCOMINGS")

# input and mapping file paths
compl_file = os.path.join(compl_dir, "compl_places_unpivoted_with_sapir_unified.xlsx")
mapping_file = os.path.join(datasets_dir, "t_General_Act_IDs_with_old_ID.xlsx")

# defining normalization function
def normalize_id(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if "_" in s:
        return s
    try:
        return str(int(float(s)))
    except:
        return s

# loading dataframes

df_compl = pd.read_excel(compl_file)

df_map = pd.read_excel(mapping_file)

# normalizing comb_id columns in both dataframes
df_compl["comb_id_norm"] = df_compl["Comb_id"].apply(normalize_id)
df_map["comb_id_norm"] = df_map["Comb_ID"].apply(normalize_id)

# extracting unique comb_id values

unique_comb = pd.DataFrame({"comb_id_norm": df_compl["comb_id_norm"].unique()})

# merging with mapping table to create fragment

fragment = unique_comb.merge(df_map, how="left", on="comb_id_norm")

# computing summary statistics

matched_unique = unique_comb["comb_id_norm"].isin(df_map["comb_id_norm"]).sum()
unmatched_unique = len(unique_comb) - matched_unique
unique_old_ids = fragment["old_id"].apply(normalize_id)
unique_old_count = unique_old_ids[unique_old_ids != ""].nunique()

# printing summary

print(f"total unique comb_id values: {len(unique_comb)}")
print(f"comb_id values with mapping: {matched_unique}")
print(f"comb_id values without mapping: {unmatched_unique}")
print(f"unique old_id matches in fragment: {unique_old_count}")

# saving the fragment to excel

output_file = os.path.join(datasets_dir, "t_General_Act_IDs_fragment_by_comb_ids.xlsx")
fragment.to_excel(output_file, index=False)
print(f"fragment saved to: {output_file} with {len(fragment)} rows")


total unique comb_id values: 624
comb_id values with mapping: 581
comb_id values without mapping: 43
unique old_id matches in fragment: 4
fragment saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/t_General_Act_IDs_fragment_by_comb_ids.xlsx with 624 rows


In [ ]:
#@title EXPORTING OLD_ID AND/OR MASTER_ID DUPLS

# base directory where mapping files are located
base_dir = "/content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/PLACES/DATASETS"

# first mapping file: t_General_Act_IDs_with_old_ID.xlsx
general_map_path = os.path.join(base_dir, "t_General_Act_IDs_with_old_ID.xlsx")
df_general = pd.read_excel(general_map_path)

# find all non-unique values in old_id and ID_master
dupe_old    = df_general[df_general.duplicated(subset="old_id",    keep=False)]["old_id"].unique()
dupe_master = df_general[df_general.duplicated(subset="ID_master", keep=False)]["ID_master"].unique()

# second mapping file: t_Master_table_draft_dana_withdates_mapped.xlsx
dana_map_path = os.path.join(base_dir, "t_Master_table_draft_dana_withdates_mapped.xlsx")
df_dana       = pd.read_excel(dana_map_path)

# filter rows where old_id or ID_master matches any non-unique value
mask = df_dana["old_id"].isin(dupe_old) | df_dana["ID_master"].isin(dupe_master)
df_duplicates = df_dana[mask]

# save filtered rows to a separate Excel for inspection
output_path = os.path.join(base_dir, "dana_mapping_duplicates.xlsx")
df_duplicates.to_excel(output_path, index=False)

print(f"Filtered rows with duplicate IDs saved to: {output_path}")

Filtered rows with duplicate IDs saved to: /content/drive/My Drive/01_עבודות_ופרויקטים/יד_ושם/UKRAINE_M.52/DATASETS/dana_mapping_duplicates.xlsx
